In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pickle
from sklearn.ensemble import RandomForestRegressor
# from rfpimp import permutation_importances
from sklearn.metrics import r2_score
from scipy.optimize import minimize
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False



# import seaborn as sns

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true))

model_folder = 'model/'
ot_model_name = 'one_time_regression_model.sav'
rec_model_name = 'recurring_monthly_regression_model.sav'
mkt_spend_model_name = 'mkt_spend_model_new_fsv_signed.sav'

import enum
# Using enum class create enumerations
class Mix(enum.Enum):
    one_time_cfo_fixed = 0
    one_time_cfo_hourly = 1
    one_time_cpaaccounting_advisory_fixed = 2
    one_time_cpaaccounting_advisory_hourly = 3
    one_time_fpa_fixed = 4
    one_time_fpa_hourly = 5
    one_time_full_charge_bookkeeping_fixed = 6
    one_time_full_charge_bookkeeping_hourly = 7
    one_time_tax_preparation_fixed = 8
    one_time_tax_preparation_hourly = 9
    recurring_monthly_cfo_fixed = 10
    recurring_monthly_cfo_hourly = 11
    recurring_monthly_cpaaccounting_advisory_fixed = 12
    recurring_monthly_cpaaccounting_advisory_hourly = 13
    recurring_monthly_fpa_fixed = 14
    recurring_monthly_fpa_hourly = 15
    recurring_monthly_full_charge_bookkeeping_fixed = 16
    recurring_monthly_full_charge_bookkeeping_hourly = 17
    recurring_monthly_tax_preparation_fixed = 18
    recurring_monthly_tax_preparation_hourly = 19
    cfo = [one_time_cfo_fixed, one_time_cfo_hourly, recurring_monthly_cfo_fixed, recurring_monthly_cfo_hourly]
    cpa = [one_time_cpaaccounting_advisory_fixed, one_time_cpaaccounting_advisory_hourly, recurring_monthly_cpaaccounting_advisory_fixed, recurring_monthly_cpaaccounting_advisory_hourly]
    fpa = [one_time_fpa_fixed, one_time_fpa_hourly, recurring_monthly_fpa_fixed, recurring_monthly_fpa_hourly]
    bkp = [one_time_full_charge_bookkeeping_fixed, one_time_full_charge_bookkeeping_hourly, recurring_monthly_full_charge_bookkeeping_fixed, recurring_monthly_full_charge_bookkeeping_hourly]
    tax = [one_time_tax_preparation_fixed, one_time_tax_preparation_hourly, recurring_monthly_tax_preparation_fixed, recurring_monthly_tax_preparation_hourly]
    fixed = [one_time_cfo_fixed, one_time_cpaaccounting_advisory_fixed, one_time_fpa_fixed, one_time_full_charge_bookkeeping_fixed, one_time_tax_preparation_fixed, 
             recurring_monthly_cfo_fixed, recurring_monthly_cpaaccounting_advisory_fixed, recurring_monthly_fpa_fixed, recurring_monthly_full_charge_bookkeeping_fixed, recurring_monthly_tax_preparation_fixed]
    hourly = [one_time_cfo_hourly, one_time_cpaaccounting_advisory_hourly, one_time_fpa_hourly, one_time_full_charge_bookkeeping_hourly, one_time_tax_preparation_hourly, 
             recurring_monthly_cfo_hourly, recurring_monthly_cpaaccounting_advisory_hourly, recurring_monthly_fpa_hourly, recurring_monthly_full_charge_bookkeeping_hourly, recurring_monthly_tax_preparation_hourly]
    price_cfo = 20
    price_cpa = 21
    price_fpa = 22
    price_bkp = 23
    price_tax = 24
    revenue_0_1 = 25
    revenue_1_10 = 26
    revenue_10_50 = 27
    revenue_5_plus = 28
    revenue_pre = 29
    revenue_no = 30
    
ot_model = pickle.load(open(f'{model_folder}{ot_model_name}', 'rb'))
rec_model = pickle.load(open(f'{model_folder}{rec_model_name}', 'rb'))
mkt_model = pickle.load(open(f'{model_folder}{mkt_spend_model_name}', 'rb'))

C:\Users\ArvindYekkirala\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ArvindYekkirala\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\ArvindYekkirala\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\ArvindYekkirala\AppData\Roaming\Python\Python37\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = pd.read_csv('pr_details_3_dec_4.csv')
data.head()
data['pr'] = data['arvinds_pr']
data['busy_season'] = data['busy_season'].astype(int)
display(data.shape)
data = data.loc[data['busy_season'] != 1]
display(data.shape)

(63957, 25)

(61727, 25)

In [3]:
avg_sow_values = [77718.41,
53898.60,
14127.43,
18679.06,
32366.43,
11326.93,
9753.38,
12150.07,
2557.83,
13771.27,
59276.17,
70145.20,
30857.58,
39150.52,
47957.14,
25765.25,
15544.92,
21439.08,
17008.33,
37121.58]

In [4]:
x_0_date = '2021-11-01'
x_0_data_date = data.loc[(data.entry_month == x_0_date) & ((data.project_frequency == 'Recurring monthly') | (data.project_frequency == 'One Time'))]
x_0_data = data.loc[((data.project_frequency == 'Recurring monthly') | (data.project_frequency == 'One Time'))]
x_0_data['level'] = x_0_data['project_frequency'] + '-' + x_0_data['service'] + '-' + x_0_data['client_rate_type']

x_0_transform = x_0_data.groupby(['level', 'entry_month']).agg({'project_id':'nunique', 'pr':'sum'}).reset_index()
x_0_transform = x_0_transform.pivot(index='entry_month', columns='level', values='pr')
x_0_transform.columns = [col.lower().replace(' ','-').replace('/','').replace('&', '').replace('-','_') for col in x_0_transform.columns]
x_0 = x_0_transform.values[0].tolist()
x_0_transform = x_0_transform.fillna(1000)

service_df=pd.DataFrame(x_0_data_date.groupby(['entry_month','service'])['client_rate_hourly'].median()).reset_index()
df = pd.pivot_table(service_df, values = ['client_rate_hourly'], index=['entry_month'], columns = 'service').reset_index()

df.columns = ['entry_month','cfo_price','cpa_price','fpa_price','book_price','tax_price']
for col in df.columns[1:]:
    df[col] = df[col].fillna(df[col].mean())
    x_0_transform[col] = df[col][0]

x_0.extend(df.values[0].tolist()[1:])

x_0_transform_rev = x_0_data.groupby(['annual_revenue', 'entry_month']).agg({'project_id':'nunique', 'pr':'sum'}).reset_index()
x_0_transform_rev = x_0_transform_rev.pivot(index='entry_month', columns='annual_revenue', values='pr')
x_0_transform_rev.columns = ['revenue_0_1' ,'revenue_1_10', 'revenue_10_50', 'revenue_5_plus', 'revenue_pre', 'revenue_no']
x_0.extend(x_0_transform_rev.values[0].tolist())

for col in x_0_transform_rev.columns:
    x_0_transform_rev[col] = x_0_transform_rev[col].fillna(1000)
    x_0_transform[col] = x_0_transform_rev[col]
# x_0_transform_rev

In [5]:
output_columns = x_0_transform.columns

In [6]:
output_frequency = ['one_time', 'one_time', 'one_time', 'one_time', 'one_time', 'one_time', 'one_time', 'one_time', 'one_time', 'one_time',
                    'recurring', 'recurring', 'recurring', 'recurring', 'recurring', 'recurring', 'recurring', 'recurring', 'recurring', 'recurring',
                   'both','both','both','both','both','both','both','both','both','both','both']
output_sl = ['cfo', 'cfo', 'cpa', 'cpa', 'fpa', 'fpa', 'bkp', 'bkp', 'tax', 'tax', 'cfo', 'cfo', 'cpa', 'cpa', 'fpa', 'fpa', 'bkp', 'bkp', 'tax', 'tax',
             'price','price','price','price','price','revenue','revenue','revenue','revenue','revenue','revenue']
output_rate_type = ['fixed', 'hourly', 'fixed', 'hourly', 'fixed', 'hourly', 'fixed', 'hourly', 'fixed', 'hourly', 'fixed', 'hourly', 'fixed', 'hourly', 'fixed', 'hourly', 'fixed', 'hourly', 'fixed', 'hourly',
                   'price','price','price','price','price','revenue','revenue','revenue','revenue','revenue','revenue']

In [7]:
def objective_fn_forecast(x):
    
    include_trend = False
    

    ot_cfo = x[Mix.one_time_cfo_fixed.value] + x[Mix.one_time_cfo_hourly.value]
    ot_cpa = x[Mix.one_time_cpaaccounting_advisory_fixed.value] + x[Mix.one_time_cpaaccounting_advisory_hourly.value]
    ot_fpa = x[Mix.one_time_fpa_fixed.value] + x[Mix.one_time_fpa_hourly.value]
    ot_bkp = x[Mix.one_time_full_charge_bookkeeping_fixed.value] + x[Mix.one_time_full_charge_bookkeeping_hourly.value]
    ot_tax = x[Mix.one_time_tax_preparation_fixed.value] + x[Mix.one_time_tax_preparation_hourly.value]
    
    price_cfo = x[Mix.price_cfo.value]
    price_cpa = x[Mix.price_cpa.value]
    price_fpa = x[Mix.price_fpa.value]
    price_bkp = x[Mix.price_bkp.value]
    price_tax = x[Mix.price_tax.value]
    
    #price values
    price_vals = [price_cfo, price_cpa, price_fpa, price_bkp, price_tax]
    
    revenue_0_1 = x[Mix.revenue_0_1.value]
    revenue_1_10 = x[Mix.revenue_1_10.value]
    revenue_10_50 = x[Mix.revenue_10_50.value]
    revenue_5_plus = x[Mix.revenue_5_plus.value]
    revenue_pre = x[Mix.revenue_pre.value]
    revenue_no = x[Mix.revenue_no.value]

    total_ot_pr = ot_cfo + ot_cpa + ot_fpa + ot_bkp + ot_tax

    #service line mix
    ot_sl_mix = [ot_cfo, ot_cpa, ot_fpa, ot_bkp, ot_tax]
    ot_sl_mix = [val/total_ot_pr for val in ot_sl_mix]
    base_value = ot_sl_mix[-1]
    ot_sl_mix = [np.log(val/base_value) for val in ot_sl_mix]

    # cfo mix
    fixed_value = np.where(x[Mix.one_time_cfo_fixed.value] <1 , 1, x[Mix.one_time_cfo_fixed.value] )
    hourly_value = np.where(x[Mix.one_time_cfo_hourly.value] <1 , 1, x[Mix.one_time_cfo_hourly.value])
    ot_cfo_mix = np.log(fixed_value/hourly_value)

    # cpa mix
    fixed_value = np.where(x[Mix.one_time_cpaaccounting_advisory_fixed.value] <1 , 1, x[Mix.one_time_cpaaccounting_advisory_fixed.value])
    hourly_value = np.where(x[Mix.one_time_cpaaccounting_advisory_hourly.value] <1 , 1, x[Mix.one_time_cpaaccounting_advisory_hourly.value])
    ot_cpa_mix = np.log(fixed_value/hourly_value)

    # fpa mix
    fixed_value = np.where(x[Mix.one_time_fpa_fixed.value] <1 , 1, x[Mix.one_time_fpa_fixed.value])
    hourly_value = np.where(x[Mix.one_time_fpa_hourly.value] <1 , 1, x[Mix.one_time_fpa_hourly.value])
    ot_fpa_mix = np.log(fixed_value/hourly_value)

    # bkp mix
    fixed_value = np.where(x[Mix.one_time_full_charge_bookkeeping_fixed.value] <1 , 1, x[Mix.one_time_full_charge_bookkeeping_fixed.value] )
    hourly_value = np.where(x[Mix.one_time_full_charge_bookkeeping_hourly.value] <1 , 1, x[Mix.one_time_full_charge_bookkeeping_hourly.value])
    ot_bkp_mix = np.log(fixed_value/hourly_value)

    # tax mix
    fixed_value = np.where(x[Mix.one_time_tax_preparation_fixed.value] <1 , 1, x[Mix.one_time_tax_preparation_fixed.value])
    hourly_value = np.where(x[Mix.one_time_tax_preparation_hourly.value] <1 , 1, x[Mix.one_time_tax_preparation_hourly.value])
    ot_tax_mix = np.log(fixed_value/hourly_value)
    
    #revenue mix
    revenue_mix = [revenue_0_1 ,revenue_1_10, revenue_10_50, revenue_5_plus, revenue_pre, revenue_no]
    revenue_mix = [val/total_ot_pr for val in revenue_mix]
    base_value = revenue_mix[-1]
    revenue_mix = [np.log(val/base_value) for val in revenue_mix]
    
    #   
    model_x = [1]
    model_x.extend(ot_sl_mix[:-1])
    if include_trend:
        model_x.extend([ot_cfo_mix, ot_cpa_mix, ot_fpa_mix, ot_bkp_mix, ot_tax_mix, total_ot_pr, trend])
    else:
        model_x.extend([ot_cfo_mix, ot_cpa_mix, ot_fpa_mix, ot_bkp_mix, ot_tax_mix, total_ot_pr])
    
    model_x.extend(price_vals)
    model_x.extend(revenue_mix)
    ot_prediction = ot_model.predict(model_x)[0]


    ## Recurring data
    rec_cfo = x[Mix.recurring_monthly_cfo_fixed.value] + x[Mix.recurring_monthly_cfo_hourly.value]
    rec_cpa = x[Mix.recurring_monthly_cpaaccounting_advisory_fixed.value] + x[Mix.recurring_monthly_cpaaccounting_advisory_hourly.value]
    rec_fpa = x[Mix.recurring_monthly_fpa_fixed.value] + x[Mix.recurring_monthly_fpa_hourly.value]
    rec_bkp = x[Mix.recurring_monthly_full_charge_bookkeeping_fixed.value] + x[Mix.recurring_monthly_full_charge_bookkeeping_hourly.value]
    rec_tax = x[Mix.recurring_monthly_tax_preparation_fixed.value] + x[Mix.recurring_monthly_tax_preparation_hourly.value]

    total_rec_pr = rec_cfo + rec_cpa + rec_fpa + rec_bkp + rec_tax

    #service line mix
    rec_sl_mix = [rec_cfo, rec_cpa, rec_fpa, rec_bkp, rec_tax]
    rec_sl_mix = [val/total_rec_pr for val in rec_sl_mix]
    base_value = rec_sl_mix[-1]
    rec_sl_mix = [np.log(val/base_value) for val in rec_sl_mix]

    # cfo mix
    fixed_value = np.where(x[Mix.recurring_monthly_cfo_fixed.value] <1 , 1, x[Mix.recurring_monthly_cfo_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_cfo_hourly.value] <1 , 1, x[Mix.recurring_monthly_cfo_hourly.value] )
    rec_cfo_mix = np.log(fixed_value/hourly_value)

    # cpa mix
    fixed_value = np.where(x[Mix.recurring_monthly_cpaaccounting_advisory_fixed.value] <1 , 1, x[Mix.recurring_monthly_cpaaccounting_advisory_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_cpaaccounting_advisory_hourly.value] <1 , 1, x[Mix.recurring_monthly_cpaaccounting_advisory_hourly.value])
    rec_cpa_mix = np.log(fixed_value/hourly_value)

    # fpa mix
    fixed_value = np.where(x[Mix.recurring_monthly_fpa_fixed.value] <1 , 1, x[Mix.recurring_monthly_fpa_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_fpa_hourly.value] <1 , 1, x[Mix.recurring_monthly_fpa_hourly.value])
    rec_fpa_mix = np.log(fixed_value/hourly_value)

    # bkp mix
    fixed_value = np.where(x[Mix.recurring_monthly_full_charge_bookkeeping_fixed.value] <1 , 1, x[Mix.recurring_monthly_full_charge_bookkeeping_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_full_charge_bookkeeping_hourly.value] <1 , 1, x[Mix.recurring_monthly_full_charge_bookkeeping_hourly.value])
    rec_bkp_mix = np.log(fixed_value/hourly_value)

    # tax mix
    fixed_value = np.where(x[Mix.recurring_monthly_tax_preparation_fixed.value] <1 , 1, x[Mix.recurring_monthly_tax_preparation_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_tax_preparation_hourly.value] <1 , 1, x[Mix.recurring_monthly_tax_preparation_hourly.value])
    rec_tax_mix = np.log(fixed_value/hourly_value)

    model_x = [1]
    model_x.extend(rec_sl_mix[:-1])
    if include_trend:
        model_x.extend([rec_cfo_mix, rec_cpa_mix, rec_fpa_mix, rec_bkp_mix, rec_tax_mix, total_rec_pr, trend])
    else:
        model_x.extend([rec_cfo_mix, rec_cpa_mix, rec_fpa_mix, rec_bkp_mix, rec_tax_mix, total_rec_pr])
    model_x.extend(price_vals)
    model_x.extend(revenue_mix)
    rec_prediction = rec_model.predict(model_x)[0]
    
    if print_mode == 1:
        print(f'One time prediction: {ot_prediction}')
        print(f'Recurring prediction: {rec_prediction}')
        print(f'Total prediction: {ot_prediction + rec_prediction}')
    return (ot_prediction, rec_prediction)

In [8]:
def objective_fn(x):
    
    include_trend = False
    

    ot_cfo = x[Mix.one_time_cfo_fixed.value] + x[Mix.one_time_cfo_hourly.value]
    ot_cpa = x[Mix.one_time_cpaaccounting_advisory_fixed.value] + x[Mix.one_time_cpaaccounting_advisory_hourly.value]
    ot_fpa = x[Mix.one_time_fpa_fixed.value] + x[Mix.one_time_fpa_hourly.value]
    ot_bkp = x[Mix.one_time_full_charge_bookkeeping_fixed.value] + x[Mix.one_time_full_charge_bookkeeping_hourly.value]
    ot_tax = x[Mix.one_time_tax_preparation_fixed.value] + x[Mix.one_time_tax_preparation_hourly.value]
    
    price_cfo = x[Mix.price_cfo.value]
    price_cpa = x[Mix.price_cpa.value]
    price_fpa = x[Mix.price_fpa.value]
    price_bkp = x[Mix.price_bkp.value]
    price_tax = x[Mix.price_tax.value]
    
    #price values
    price_vals = [price_cfo, price_cpa, price_fpa, price_bkp, price_tax]
    
    revenue_0_1 = x[Mix.revenue_0_1.value]
    revenue_1_10 = x[Mix.revenue_1_10.value]
    revenue_10_50 = x[Mix.revenue_10_50.value]
    revenue_5_plus = x[Mix.revenue_5_plus.value]
    revenue_pre = x[Mix.revenue_pre.value]
    revenue_no = x[Mix.revenue_no.value]

    total_ot_pr = ot_cfo + ot_cpa + ot_fpa + ot_bkp + ot_tax

    #service line mix
    ot_sl_mix = [ot_cfo, ot_cpa, ot_fpa, ot_bkp, ot_tax]
    ot_sl_mix = [val/total_ot_pr for val in ot_sl_mix]
    base_value = ot_sl_mix[-1]
    ot_sl_mix = [np.log(val/base_value) for val in ot_sl_mix]

    # cfo mix
    fixed_value = np.where(x[Mix.one_time_cfo_fixed.value] <1 , 1, x[Mix.one_time_cfo_fixed.value] )
    hourly_value = np.where(x[Mix.one_time_cfo_hourly.value] <1 , 1, x[Mix.one_time_cfo_hourly.value])
    ot_cfo_mix = np.log(fixed_value/hourly_value)

    # cpa mix
    fixed_value = np.where(x[Mix.one_time_cpaaccounting_advisory_fixed.value] <1 , 1, x[Mix.one_time_cpaaccounting_advisory_fixed.value])
    hourly_value = np.where(x[Mix.one_time_cpaaccounting_advisory_hourly.value] <1 , 1, x[Mix.one_time_cpaaccounting_advisory_hourly.value])
    ot_cpa_mix = np.log(fixed_value/hourly_value)

    # fpa mix
    fixed_value = np.where(x[Mix.one_time_fpa_fixed.value] <1 , 1, x[Mix.one_time_fpa_fixed.value])
    hourly_value = np.where(x[Mix.one_time_fpa_hourly.value] <1 , 1, x[Mix.one_time_fpa_hourly.value])
    ot_fpa_mix = np.log(fixed_value/hourly_value)

    # bkp mix
    fixed_value = np.where(x[Mix.one_time_full_charge_bookkeeping_fixed.value] <1 , 1, x[Mix.one_time_full_charge_bookkeeping_fixed.value] )
    hourly_value = np.where(x[Mix.one_time_full_charge_bookkeeping_hourly.value] <1 , 1, x[Mix.one_time_full_charge_bookkeeping_hourly.value])
    ot_bkp_mix = np.log(fixed_value/hourly_value)

    # tax mix
    fixed_value = np.where(x[Mix.one_time_tax_preparation_fixed.value] <1 , 1, x[Mix.one_time_tax_preparation_fixed.value])
    hourly_value = np.where(x[Mix.one_time_tax_preparation_hourly.value] <1 , 1, x[Mix.one_time_tax_preparation_hourly.value])
    ot_tax_mix = np.log(fixed_value/hourly_value)
    
    #revenue mix
    revenue_mix = [revenue_0_1 ,revenue_1_10, revenue_10_50, revenue_5_plus, revenue_pre, revenue_no]
    revenue_mix = [val/total_ot_pr for val in revenue_mix]
    base_value = revenue_mix[-1]
    revenue_mix = [np.log(val/base_value) for val in revenue_mix]
    
    #   
    model_x = [1]
    model_x.extend(ot_sl_mix[:-1])
    if include_trend:
        model_x.extend([ot_cfo_mix, ot_cpa_mix, ot_fpa_mix, ot_bkp_mix, ot_tax_mix, total_ot_pr, trend])
    else:
        model_x.extend([ot_cfo_mix, ot_cpa_mix, ot_fpa_mix, ot_bkp_mix, ot_tax_mix, total_ot_pr])
    
    model_x.extend(price_vals)
    model_x.extend(revenue_mix)
    ot_prediction = ot_model.predict(model_x)[0]


    ## Recurring data
    rec_cfo = x[Mix.recurring_monthly_cfo_fixed.value] + x[Mix.recurring_monthly_cfo_hourly.value]
    rec_cpa = x[Mix.recurring_monthly_cpaaccounting_advisory_fixed.value] + x[Mix.recurring_monthly_cpaaccounting_advisory_hourly.value]
    rec_fpa = x[Mix.recurring_monthly_fpa_fixed.value] + x[Mix.recurring_monthly_fpa_hourly.value]
    rec_bkp = x[Mix.recurring_monthly_full_charge_bookkeeping_fixed.value] + x[Mix.recurring_monthly_full_charge_bookkeeping_hourly.value]
    rec_tax = x[Mix.recurring_monthly_tax_preparation_fixed.value] + x[Mix.recurring_monthly_tax_preparation_hourly.value]

    total_rec_pr = rec_cfo + rec_cpa + rec_fpa + rec_bkp + rec_tax

    #service line mix
    rec_sl_mix = [rec_cfo, rec_cpa, rec_fpa, rec_bkp, rec_tax]
    rec_sl_mix = [val/total_rec_pr for val in rec_sl_mix]
    base_value = rec_sl_mix[-1]
    rec_sl_mix = [np.log(val/base_value) for val in rec_sl_mix]

    # cfo mix
    fixed_value = np.where(x[Mix.recurring_monthly_cfo_fixed.value] <1 , 1, x[Mix.recurring_monthly_cfo_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_cfo_hourly.value] <1 , 1, x[Mix.recurring_monthly_cfo_hourly.value] )
    rec_cfo_mix = np.log(fixed_value/hourly_value)

    # cpa mix
    fixed_value = np.where(x[Mix.recurring_monthly_cpaaccounting_advisory_fixed.value] <1 , 1, x[Mix.recurring_monthly_cpaaccounting_advisory_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_cpaaccounting_advisory_hourly.value] <1 , 1, x[Mix.recurring_monthly_cpaaccounting_advisory_hourly.value])
    rec_cpa_mix = np.log(fixed_value/hourly_value)

    # fpa mix
    fixed_value = np.where(x[Mix.recurring_monthly_fpa_fixed.value] <1 , 1, x[Mix.recurring_monthly_fpa_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_fpa_hourly.value] <1 , 1, x[Mix.recurring_monthly_fpa_hourly.value])
    rec_fpa_mix = np.log(fixed_value/hourly_value)

    # bkp mix
    fixed_value = np.where(x[Mix.recurring_monthly_full_charge_bookkeeping_fixed.value] <1 , 1, x[Mix.recurring_monthly_full_charge_bookkeeping_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_full_charge_bookkeeping_hourly.value] <1 , 1, x[Mix.recurring_monthly_full_charge_bookkeeping_hourly.value])
    rec_bkp_mix = np.log(fixed_value/hourly_value)

    # tax mix
    fixed_value = np.where(x[Mix.recurring_monthly_tax_preparation_fixed.value] <1 , 1, x[Mix.recurring_monthly_tax_preparation_fixed.value])
    hourly_value = np.where(x[Mix.recurring_monthly_tax_preparation_hourly.value] <1 , 1, x[Mix.recurring_monthly_tax_preparation_hourly.value])
    rec_tax_mix = np.log(fixed_value/hourly_value)

    model_x = [1]
    model_x.extend(rec_sl_mix[:-1])
    if include_trend:
        model_x.extend([rec_cfo_mix, rec_cpa_mix, rec_fpa_mix, rec_bkp_mix, rec_tax_mix, total_rec_pr, trend])
    else:
        model_x.extend([rec_cfo_mix, rec_cpa_mix, rec_fpa_mix, rec_bkp_mix, rec_tax_mix, total_rec_pr])
    model_x.extend(price_vals)
    model_x.extend(revenue_mix)
    rec_prediction = rec_model.predict(model_x)[0]
    
    if print_mode == 1:
        print(f'One time prediction: {ot_prediction}')
        print(f'Recurring prediction: {rec_prediction}')
        print(f'Total prediction: {ot_prediction + rec_prediction}')
    return -1*0.001*0.001*(ot_prediction + rec_prediction)

In [9]:
## Constraints
one_time_mix_lower = 0.09
one_time_mix_upper = 0.2

recurring_mix_lower = 0.5
recurring_mix_upper = 0.9

cfo_mix_lower = 0.04
cfo_mix_upper = 0.25

cpa_mix_lower = 0.15
cpa_mix_upper = 0.4

fpa_mix_lower = 0.03
fpa_mix_upper = 0.15

bkp_mix_lower = 0.2
bkp_mix_upper = 0.5

tax_mix_lower = 0.05
tax_mix_upper = 0.15

fixed_mix_lower = 0.3
fixed_mix_upper = 0.7

hourly_mix_lower = 0.15
hourly_mix_upper = 0.5

#Realization ratios
current_month = 0.45
previous_month = 0.26
two_months_back = 0.03
three_months_back = 0.01

#
churn_rate = 0.89

fsv_ot_start_percent = 0.7

target_achieve_factor = 0.9
target_miss_factor = 0.8


In [28]:
#Reduction of salesrep

fsv_target_list = [5835000,5820000,5850000,5700000,5550000,5550000,5550000,5400000,5400000,5250000,5100000,5100000]


In [34]:
#x_0_dates = ['2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01']

#x_0_dates = [ '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01']
x_0_dates = [ '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', 
             '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01']

# x_0_dates = [ '2022-01-01']

SALES_REP_INCR = False
SALES_REP_INCR_MONTH = 1
SALES_REP_INCR_COUNT = 1
SALES_REP_INCR_TGT = 125000

#
churn_rate = 0.894

#Realization ratios
current_month = 0.6
previous_month = 0.25
two_months_back = 0.03
three_months_back = 0.01

#fsv_targets = [3984375, 4228125, 4462500, 4659375, 4893750, 5253125, 5678125]
#fsv_targets = [ 4462500, 4659375, 4893750, 5253125, 5678125]
#fsv_targets = [4000000, 4500000, 5000000, 5500000, 6000000]
# fsv_targets = [5000000, 5500000, 6000000]

fsv_targets = [5000000, 5250000, 5500000, 5500000, 5750000, 6000000, 6250000,6500000,6750000, 7000000, 7250000, 7500000, 7750000]
# fsv_targets = [fsv - 200000 for fsv in fsv_targets]

final_index_vals = ['x_0_date','fsv_target','total_fsv_signed','ot_fsv_signed','rec_fsv_signed','total_fsv_launched','new_ot_fsv','new_rec_fsv',
                    'carryover_ot_ir','carryover_rec_ir','new_ot_ir','new_rec_ir','ot_mix','rec_mix','cfo_mix','cpa_mix','fpa_mix','bkp_mix','tax_mix','fixed_mix','hourly_mix','onetime_cfo_fixed_mix','onetime_cfo_hourly_mix',
                   'onetime_cpa_fixed_mix','onetime_cpa_hourly_mix','onetime_fpa_fixed_mix','onetime_fpa_hourly_mix','onetime_bkp_fixed_mix','onetime_bkp_hourly_mix','onetime_tax_fixed_mix','onetime_tax_hourly_mix',
                   'recurring_cfo_fixed_mix','recurring_cfo_hourly_mix','recurring_cpa_fixed_mix','recurring_cpa_hourly_mix','recurring_fpa_fixed_mix','recurring_fpa_hourly_mix','recurring_bkp_fixed_mix','recurring_bkp_hourly_mix','recurring_tax_fixed_mix','recurring_tax_hourly_mix',
                   'cfo_onetime_mix','cpa_onetime_mix','fpa_onetime_mix','bkp_onetime_mix','tax_onetime_mix',
                 'cfo_recurring_mix','cpa_recurring_mix','fpa_recurring_mix','bkp_recurring_mix','tax_recurring_mix',
                   'cfo_of_onetime','cfo_of_recurring','cpa_of_onetime','cpa_of_recurring','fpa_of_onetime','fpa_of_recurring','bkp_of_onetime',
                 'bkp_of_recurring','tax_of_onetime','tax_of_recurring',
                   'cfo_fixed_mix','cfo_hourly_mix','cpa_fixed_mix','cpa_hourly_mix','fpa_fixed_mix','fpa_hourly_mix',
                 'bkp_fixed_mix','bkp_hourly_mix','tax_fixed_mix','tax_hourly_mix',
                   'annual_revenue_0_1_mix','annual_revenue_1_10_mix','annual_revenue_10_50_mix','annual_revenue_5_plus_mix',
                 'annual_revenue_pre_increment_mix','annual_revenue_revenue_no_increment_mix']
final_result = pd.DataFrame(columns=['record_date'], index = final_index_vals)
prev_records = []

for idx, value in enumerate(x_0_dates):
    x_0_date = value
    orig_idx = idx
    fsv_target = fsv_targets[idx]
    
   # if x_0_date in ['2022-01-01','2022-02-01', '2022-03-01', '2022-04-01']:
   #     tax_mix_upper = 0.45
   #     tax_mix_lower = 0.20
        
   #     one_time_mix_lower = 0.20
   #     one_time_mix_upper = 0.40

    #    recurring_mix_lower = 0.3
    #    recurring_mix_upper = 0.7
    #else : 
     #   tax_mix_upper = 0.10
     #   tax_mix_lower = 0.05
        
      #  one_time_mix_lower = 0.09
      #  one_time_mix_upper = 0.15

      #  recurring_mix_lower = 0.5
      #  recurring_mix_upper = 0.8
    
    if SALES_REP_INCR and idx >= SALES_REP_INCR_MONTH:
        fsv_target = fsv_target + (idx + 1 - SALES_REP_INCR_MONTH)*SALES_REP_INCR_COUNT*SALES_REP_INCR_TGT
    
    
    # marketting model
    inbound = 0.58
    online_advertising = 0.15
    outbound = 0.23
    partnerships = 0.04
    
    channel_mix = [inbound,online_advertising,outbound,partnerships]
    base_channel = partnerships
    channel_mix = [np.log(val/base_channel) for val in channel_mix]
    #avg_3_month = 900000 * (1.02**idx)
    avg_3_month = 1000000 * (0.99**idx)
    
    #fsv_target = np.exp(mkt_model.predict([1,avg_3_month,channel_mix[0],channel_mix[1],channel_mix[2]]))[0]
    fsv_target = fsv_target_list[idx]
    print(fsv_target)
    
    # marketting model
    
    x_0 = x_0_transform.filter(like = x_0_date, axis=0).values[0].tolist()
    x_0 = [0 if np.isnan(x) else x for x in x_0]
    
    #if idx <3:
    #    churn_rate = 0.88
    #else:
    #    churn_rate = 0.84
    
#     if idx == 0:
#         x_0_orig = [0]*len(x_0)
#     else:
    x_0_orig = x_0
    
   ### #Replace churn rate to 0.93 ###
    
    if idx > 0:
        for i in range(0,idx):
            if i <0:
                continue
            exp_factor = idx - i
            if exp_factor == 1:
                x_0 = [(x_0[ix] + prev_records[i][ix]) if ix <10 else (x_0[ix] + (churn_rate**exp_factor)*prev_records[i][ix]) for ix,value in enumerate(x_0)]
            else:
                x_0 = [(x_0[ix]) if ix <10 else (x_0[ix] + (churn_rate**exp_factor)*prev_records[i][ix]) for ix,value in enumerate(x_0)]
            
            if exp_factor == 1:
                x_0 = [(x_0[ix] + prev_records[i][ix]*previous_month/current_month) for ix,value in enumerate(x_0)]
            elif exp_factor == 2:
                x_0 = [(x_0[ix] + prev_records[i][ix]*two_months_back/current_month) for ix,value in enumerate(x_0)]
            elif exp_factor == 3:
                x_0 = [(x_0[ix] + prev_records[i][ix]*three_months_back/current_month) for ix,value in enumerate(x_0)]
    
#     # adding in projects already launched
#     x_0 = [(x_0[idx] + july_additions[idx]) if idx <10 else (x_0[idx] + (churn_rate**2)*june_additions[idx] + (churn_rate)*july_additions[idx]) for idx,value in enumerate(x_0) ]

#     # adding projects not yet launched
#     x_0 = [(x_0[idx] + june_additions[idx]*two_months_back/current_month + july_additions[idx]*previous_month/current_month) for idx,value in enumerate(x_0) ]

    
    
    # fsv available to start in current month
    fsv_to_start = current_month*fsv_target 
    
    #fsv multipled by target % achieved
    fsv_to_start = fsv_to_start*target_achieve_factor
    
    # one time limit
    upper_limit_one_time_pr_lower = fsv_to_start*one_time_mix_lower
    
    def total_fsv_constraint(x):
        ot_addition = sum(x[0:10]) - sum(x_0[0:10])
        rec_addition = sum(x[10:]) - sum(x_0[10:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12
    
        return fsv_to_start - (ot_annual_fsv + rec_annual_fsv)

    def total_fsv_constraint_lower(x):
        ot_addition = sum(x[0:10]) - sum(x_0[0:10])
        rec_addition = sum(x[10:]) - sum(x_0[10:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12

        return (ot_annual_fsv + rec_annual_fsv) - target_miss_factor*fsv_to_start

    def one_time_fsv_constraint(x):
        ot_addition = sum(x[0:10]) - sum(x_0[0:10])
        rec_addition = sum(x[10:]) - sum(x_0[10:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12

        return (fsv_to_start)*one_time_mix_upper - ot_annual_fsv

    def one_time_fsv_constraint_lower(x):
        ot_addition = sum(x[0:10]) - sum(x_0[0:10])
        rec_addition = sum(x[10:]) - sum(x_0[10:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12

        return ot_annual_fsv - (fsv_to_start)*one_time_mix_lower 

    def hourly_fsv_constraint(x):
        ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
        rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12


        return fsv_to_start*hourly_mix_upper - (ot_annual_fsv + rec_annual_fsv)

    def hourly_fsv_constraint_lower(x):

        current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
        additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper

        ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
        rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12


        return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*hourly_mix_lower

    def cfo_fsv_constraint(x):
        ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
        rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12

        return fsv_to_start*cfo_mix_upper - (ot_annual_fsv + rec_annual_fsv)

    def cfo_fsv_constraint_lower(x):
        ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
        rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12

        return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*cfo_mix_lower

    def fpa_fsv_constraint(x):
        ot_addition = sum(np.array(x)[Mix.fpa.value][0:2]) - sum(np.array(x_0)[Mix.fpa.value][0:2])
        rec_addition = sum(np.array(x)[Mix.fpa.value][2:]) - sum(np.array(x_0)[Mix.fpa.value][2:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12

        return fsv_to_start*fpa_mix_upper - (ot_annual_fsv + rec_annual_fsv)

    def tax_fsv_constraint(x):
        ot_addition = sum(np.array(x)[Mix.tax.value][0:2]) - sum(np.array(x_0)[Mix.tax.value][0:2])
        rec_addition = sum(np.array(x)[Mix.tax.value][2:]) - sum(np.array(x_0)[Mix.tax.value][2:])

        ot_annual_fsv = ot_addition/fsv_ot_start_percent
        rec_annual_fsv = rec_addition*12

        return fsv_to_start*tax_mix_upper - (ot_annual_fsv + rec_annual_fsv)  

    total_fsv_cons = {'type':'ineq', 'fun':total_fsv_constraint}
    total_fsv_cons_lower = {'type':'ineq', 'fun':total_fsv_constraint_lower}
    ot_fsv_cons = {'type':'ineq', 'fun':one_time_fsv_constraint}
    ot_fsv_cons_lower = {'type':'ineq', 'fun':one_time_fsv_constraint_lower}

    hourly_fsv_cons = {'type':'ineq', 'fun':hourly_fsv_constraint}
    hourly_fsv_cons_lower = {'type':'ineq', 'fun':hourly_fsv_constraint_lower}
    cfo_fsv_cons = {'type':'ineq', 'fun':cfo_fsv_constraint}
    cfo_fsv_cons_lower = {'type':'ineq', 'fun':cfo_fsv_constraint_lower}
    fpa_fsv_cons = {'type':'ineq', 'fun':fpa_fsv_constraint}
    tax_fsv_cons = {'type':'ineq', 'fun':tax_fsv_constraint}

    constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, ot_fsv_cons_lower, hourly_fsv_cons, hourly_fsv_cons_lower, cfo_fsv_cons, cfo_fsv_cons_lower, fpa_fsv_cons, tax_fsv_cons ]
    
    bounds = ()
    for idx, val in enumerate(x_0):
        if idx in [20,21,22,23,24]:
            bound_val = (x_0[idx]*1.05, x_0[idx]*1.15)
        elif idx in [2]:
            bound_val = (x_0[idx]*1.12, x_0[idx]*1.40 + 100000)
        elif idx in [3]:
            bound_val = (x_0[idx]*1.1, x_0[idx]*1.35 + 100000)
        elif idx in [6]:
            bound_val = (x_0[idx]*1.07, x_0[idx]*1.25 + 100000)
        elif idx in [7]:
            bound_val = (x_0[idx]*1.1, x_0[idx]*1.30 + 100000)
        elif idx in [0]:
            bound_val = (x_0[idx]*0.93, x_0[idx]*1.03 + 100000)
        elif idx in [1]:
            bound_val = (x_0[idx]*0.95, x_0[idx]*1.05 + 100000)
        elif idx in [4,5,8,9]:
            bound_val = (x_0[idx]*1.05, x_0[idx]*1.25+1000)
        elif idx in [10, 11]:
            if x_0_date in ['2022-01-01','2022-02-01', '2022-03-01']:
                #bound_val = (x_0[idx]*1.05, x_0[idx]*1.1+100)
                #bound_val = (x_0[idx]*1.05, x_0[idx]*1.15+100)
                #bound_val = (x_0[idx]*1.05, x_0[idx]*1.15+100)
                bound_val = (x_0[idx]*1 + 1 , x_0[idx] + 30000)
            else : 
                #bound_val = (x_0[idx]*1.05, x_0[idx]*1.15+100)
                bound_val = (x_0[idx]*1 + 1, x_0[idx] + 30000)
        else : 
            bound_val = (x_0[idx]*1.05, 1000000)
        #elif idx in [0, 1]:
        #    if x_0_date in ['2022-01-01','2022-02-01', '2022-03-01']:
        #        bound_val = (x_0[idx]*1.05, x_0[idx]*1.5+100)
                #bound_val = (x_0[idx]*1.05, x_0[idx] + 10000)
                #bound_val = (x_0[idx]*1.07, x_0[idx] + 50000)
         #   else : 
         #       bound_val = (x_0[idx]*1.05, x_0[idx]*1.15+100)
                #bound_val = (x_0[idx]*1.05, x_0[idx] + 10000)
                #bound_val = (x_0[idx]*1.07, x_0[idx] + 50000)
        #elif idx in [8, 9]:
        #    if x_0_date in ['2022-01-01','2022-02-01', '2022-03-01','2022-04-01']:
        #        bound_val = (x_0[idx]*1.27, 10000000)
        #    else : 
        #        bound_val = (x_0[idx]*1.05, 10000000)
        #else:
        #    if x_0_date in ['2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01']:
        #        bound_val = (x_0[idx]*1.05, 10000000)
        #        #bound_val = (x_0[idx]*1.05, x_0[idx]*1.35+100)
        #    else:
        #        bound_val = (x_0[idx]*1.05, 10000000)
        #        #bound_val = (x_0[idx]*1.05, x_0[idx]*1.35+100)
        bounds = bounds + (bound_val,)

    trend = 41
    print_mode = 0
    opt_method = 'SLSQP'
    sol = minimize(objective_fn, x_0, method=opt_method, bounds = bounds, constraints=constraints, options={'maxiter': 1000,'eps' : 0.5})
    if sol.success:
        print(f'Optimization for the month of \'{x_0_date}\' is successful')
    else:
        print(f'Optimization for the month of \'{x_0_date}\' failed')
    
    
    output_df = pd.DataFrame([sol.x, x_0, x_0_orig]).T
    output_df.columns = ['Optimized', 'Original', 'Month_Orig']
    output_df['Optimized'] = output_df.apply(lambda x: x.Optimized if x.Optimized > x.Original else x.Original, axis=1)
    output_df['increment'] = output_df['Optimized'] - output_df['Original']
    output_df['type'] = output_columns
    output_df['frequency'] = output_frequency
    output_df['sl'] = output_sl
    output_df['rate'] = output_rate_type
    month_additions = list(output_df['increment'])
    
    ot_fsv_to_start = (sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/fsv_ot_start_percent
    rec_fsv_to_start = (sum(list(output_df.Optimized)[10:20]) - sum(list(output_df.Original)[10:20]))*12
    total_fsv_to_start = ot_fsv_to_start + rec_fsv_to_start

    (ot_opt, rec_opt) = objective_fn_forecast(output_df.Optimized)
    (ot, rec) = objective_fn_forecast(output_df.Month_Orig)
    (ot_new, rec_new) = objective_fn_forecast(output_df.Original)
    
#     print(f'ot_new: {ot_opt}, rec_new: {rec_opt}')
#     print(f'ot_fsv_to_start: {ot}, rec_fsv_to_start: {rec}')

    total_fsv_signed = total_fsv_to_start/current_month
    ot_fsv_signed = ot_fsv_to_start/current_month
    rec_fsv_signed = rec_fsv_to_start/current_month
    
    
    ##Can be removed ## 
    total_fsv_launched = 1.07*(sum(list(output_df.Optimized)))
    ##Can be removed ## 
    
    
    new_ot_fsv = ot_fsv_to_start*fsv_ot_start_percent
    new_rec_fsv = rec_fsv_to_start/12
    
    if idx == 0:
        carryover_ot_ir = 0
        carryover_rec_ir = 0

        new_ot_ir = ot_opt - ot
        new_rec_ir = (rec_opt - rec)
    else:
        carryover_ot_ir = ot_new - ot
        carryover_rec_ir = rec_new - rec

        new_ot_ir = ot_opt - ot_new
        if orig_idx <3:
            new_rec_ir = 0.9*(0.95**idx)*(rec_opt - rec_new)
        else:
            new_rec_ir = 0.9*(0.95**idx)*(rec_opt - rec_new)
        #new_rec_ir = rec_opt - rec_new

#     print(f'Total FSV signed for the month: {total_fsv_to_start/current_month}')
#     print(f'Total OT FSV signed for the month: {ot_fsv_to_start/current_month}')
#     print(f'Total Rec FSV signed for the month: {rec_fsv_to_start/current_month}')
#     print('--------------------------------------------------------------')
#     print(f'Total FSV launched for the month: {1.07*(sum(list(output_df.Optimized)))}')
#     # print(f'Total OT FSV launched for the month: {ot_fsv_to_start}')
#     # print(f'Total Rec FSV launched for the month: {rec_fsv_to_start}')
#     print('--------------------------------------------------------------')
#     print(f'New OT FSV applicable for month: {ot_fsv_to_start*fsv_ot_start_percent}')
#     print(f'New Rec FSV applicable for month: {rec_fsv_to_start/12}')
#     print('--------------------------------------------------------------')
#     print(f'Carryover OT IR for month: {ot_new - ot}')
#     print(f'Carryover Rec IR applicable for month: {(rec_new - rec)}')
#     print('--------------------------------------------------------------')
#     print(f'New OT IR for month: {ot_opt - ot_new}')
#     print(f'New Rec IR applicable for month: {0.525*(rec_opt - rec_new)}')
#     print('--------------------------------------------------------------')


    cfo_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
    cpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
    fpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
    bkp_fsv_to_start = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
    tax_fsv_to_start = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12

    
    cfo_fsv_to_start_onetime = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent)
    cpa_fsv_to_start_onetime = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent)
    fpa_fsv_to_start_onetime = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent)
    bkp_fsv_to_start_onetime = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent)
    tax_fsv_to_start_onetime = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent)
    
    cfo_fsv_to_start_recurring = (sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
    cpa_fsv_to_start_recurring = (sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
    fpa_fsv_to_start_recurring = (sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
    bkp_fsv_to_start_recurring = (sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
    tax_fsv_to_start_recurring = (sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12
    
    
    
    cfo_fixed_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='cfo')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='cfo')]['Original'].sum())/fsv_ot_start_percent
    cfo_hourly_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='cfo')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='cfo')]['Original'].sum())/fsv_ot_start_percent
    cpa_fixed_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='cpa')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='cpa')]['Original'].sum())/fsv_ot_start_percent
    cpa_hourly_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='cpa')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='cpa')]['Original'].sum())/fsv_ot_start_percent
    fpa_fixed_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='fpa')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='fpa')]['Original'].sum())/fsv_ot_start_percent
    fpa_hourly_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='fpa')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='fpa')]['Original'].sum())/fsv_ot_start_percent
    bkp_fixed_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='bkp')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='bkp')]['Original'].sum())/fsv_ot_start_percent
    bkp_hourly_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='bkp')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='bkp')]['Original'].sum())/fsv_ot_start_percent
    tax_fixed_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='tax')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='tax')]['Original'].sum())/fsv_ot_start_percent
    tax_hourly_fsv_to_start_onetime = (output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='tax')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='hourly') & (output_df['sl']=='tax')]['Original'].sum())/fsv_ot_start_percent


    cfo_fixed_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='cfo')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='cfo')]['Original'].sum()) * 12
    cfo_hourly_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='cfo')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='cfo')]['Original'].sum()) * 12
    cpa_fixed_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='cpa')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='cpa')]['Original'].sum()) * 12
    cpa_hourly_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='cpa')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='cpa')]['Original'].sum()) * 12
    fpa_fixed_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='fpa')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='fpa')]['Original'].sum()) * 12
    fpa_hourly_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='fpa')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='fpa')]['Original'].sum()) * 12
    bkp_fixed_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='bkp')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='bkp')]['Original'].sum()) * 12
    bkp_hourly_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='bkp')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='bkp')]['Original'].sum()) * 12
    tax_fixed_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='tax')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='fixed') & (output_df['sl']=='tax')]['Original'].sum()) * 12
    tax_hourly_fsv_to_start_recurring = (output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='tax')]['Optimized'].sum()
                                    - output_df.loc[(output_df['frequency']=='recurring') & (output_df['rate']=='hourly') & (output_df['sl']=='tax')]['Original'].sum()) * 12

    
    cfo_fixed_fsv_to_start = cfo_fixed_fsv_to_start_onetime + cfo_fixed_fsv_to_start_recurring
    cfo_hourly_fsv_to_start = cfo_hourly_fsv_to_start_onetime + cfo_hourly_fsv_to_start_recurring
    cpa_fixed_fsv_to_start = cpa_fixed_fsv_to_start_onetime + cpa_fixed_fsv_to_start_recurring
    cpa_hourly_fsv_to_start = cpa_hourly_fsv_to_start_onetime + cpa_hourly_fsv_to_start_recurring
    fpa_fixed_fsv_to_start = fpa_fixed_fsv_to_start_onetime + fpa_fixed_fsv_to_start_recurring
    fpa_hourly_fsv_to_start = fpa_hourly_fsv_to_start_onetime + fpa_hourly_fsv_to_start_recurring
    bkp_fixed_fsv_to_start = bkp_fixed_fsv_to_start_onetime + bkp_fixed_fsv_to_start_recurring
    bkp_hourly_fsv_to_start = bkp_hourly_fsv_to_start_onetime + bkp_hourly_fsv_to_start_recurring
    tax_fixed_fsv_to_start = tax_fixed_fsv_to_start_onetime + tax_fixed_fsv_to_start_recurring
    tax_hourly_fsv_to_start = tax_hourly_fsv_to_start_onetime + tax_hourly_fsv_to_start_recurring
    
    
    total_anual_revenue_increment = output_df.loc[(output_df['sl']=='revenue')]['increment'].sum()
    annual_revenue_0_1_increment =  output_df.loc[(output_df['type']=='revenue_0_1')]['increment'].sum()
    annual_revenue_1_10_increment =  output_df.loc[(output_df['type']=='revenue_1_10')]['increment'].sum()
    annual_revenue_10_50_increment =  output_df.loc[(output_df['type']=='revenue_10_50')]['increment'].sum()
    annual_revenue_5_plus_increment =  output_df.loc[(output_df['type']=='revenue_5_plus')]['increment'].sum()
    annual_revenue_pre_increment =  output_df.loc[(output_df['type']=='revenue_pre')]['increment'].sum()
    annual_revenue_revenue_no_increment =  output_df.loc[(output_df['type']=='revenue_no')]['increment'].sum()

    #cfo_onetime_fsv_to_start =  (output_df.loc[(output_df['frequency']=='one_time') & (output_df['sl']=='cfo')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='one_time')  & (output_df['sl']=='cfo')]['Original'].sum())/fsv_ot_start_percent
    #cfo_recurring_fsv_to_start =  (output_df.loc[(output_df['frequency']=='recurring') & (output_df['sl']=='cfo')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='recurring')  & (output_df['sl']=='cfo')]['Original'].sum())* 12
    #cpa_onetime_fsv_to_start =  (output_df.loc[(output_df['frequency']=='one_time') & (output_df['sl']=='cpa')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='one_time')  & (output_df['sl']=='cpa')]['Original'].sum())/fsv_ot_start_percent
    #cpa_recurring_fsv_to_start =  (output_df.loc[(output_df['frequency']=='recurring') & (output_df['sl']=='cpa')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='recurring')  & (output_df['sl']=='cpa')]['Original'].sum())* 12
    #fpa_onetime_fsv_to_start =  (output_df.loc[(output_df['frequency']=='one_time') & (output_df['sl']=='fpa')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='one_time')  & (output_df['sl']=='fpa')]['Original'].sum())/fsv_ot_start_percent
    #fpa_recurring_fsv_to_start =  (output_df.loc[(output_df['frequency']=='recurring') & (output_df['sl']=='fpa')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='recurring')  & (output_df['sl']=='fpa')]['Original'].sum())* 12
    #bkp_onetime_fsv_to_start =  (output_df.loc[(output_df['frequency']=='one_time') & (output_df['sl']=='bkp')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='one_time')  & (output_df['sl']=='bkp')]['Original'].sum())/fsv_ot_start_percent
    #bkp_recurring_fsv_to_start =  (output_df.loc[(output_df['frequency']=='recurring') & (output_df['sl']=='bkp')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='recurring')  & (output_df['sl']=='bkp')]['Original'].sum())* 12
    #tax_onetime_fsv_to_start =  (output_df.loc[(output_df['frequency']=='one_time') & (output_df['sl']=='tax')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='one_time')  & (output_df['sl']=='tax')]['Original'].sum())/fsv_ot_start_percent
    #tax_recurring_fsv_to_start =  (output_df.loc[(output_df['frequency']=='recurring') & (output_df['sl']=='tax')]['Optimized'].sum()
    #                                - output_df.loc[(output_df['frequency']=='recurring')  & (output_df['sl']=='tax')]['Original'].sum())* 12





    
    #print(sum(list(output_df.Original[Mix.cfo.value])[10:11]))
    #print(cfo_hourly_fsv_to_start)
    #print(cfo_fsv_to_start)
    

    fixed_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fixed.value])[0:5]) - sum(list(output_df.Original[Mix.fixed.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fixed.value])[5:]) - sum(list(output_df.Original[Mix.fixed.value])[5:]))*12
    hourly_fsv_to_start = ((sum(list(output_df.Optimized[Mix.hourly.value])[0:5]) - sum(list(output_df.Original[Mix.hourly.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.hourly.value])[5:]) - sum(list(output_df.Original[Mix.hourly.value])[5:]))*12

    
    ot_mix = int(np.round(ot_fsv_to_start*100/total_fsv_to_start))
    rec_mix = int(np.round(rec_fsv_to_start*100/total_fsv_to_start))
    
    cfo_mix = int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))
    cpa_mix = int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))
    fpa_mix = int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))
    bkp_mix = int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))
    tax_mix = int(np.round(tax_fsv_to_start*100/total_fsv_to_start))
    
    onetime_cfo_fixed_mix = int(np.round(cfo_fixed_fsv_to_start_onetime*100/cfo_fsv_to_start_onetime))
    onetime_cfo_hourly_mix = int(np.round(cfo_hourly_fsv_to_start_onetime*100/cfo_fsv_to_start_onetime))
    onetime_cpa_fixed_mix = int(np.round(cpa_fixed_fsv_to_start_onetime*100/cpa_fsv_to_start_onetime))
    onetime_cpa_hourly_mix = int(np.round(cpa_hourly_fsv_to_start_onetime*100/cpa_fsv_to_start_onetime))
    onetime_fpa_fixed_mix = int(np.round(fpa_fixed_fsv_to_start_onetime*100/fpa_fsv_to_start_onetime))
    onetime_fpa_hourly_mix = int(np.round(fpa_hourly_fsv_to_start_onetime*100/fpa_fsv_to_start_onetime))
    onetime_bkp_fixed_mix = int(np.round(bkp_fixed_fsv_to_start_onetime*100/bkp_fsv_to_start_onetime))
    onetime_bkp_hourly_mix = int(np.round(bkp_hourly_fsv_to_start_onetime*100/bkp_fsv_to_start_onetime))
    onetime_tax_fixed_mix = int(np.round(tax_fixed_fsv_to_start_onetime*100/tax_fsv_to_start_onetime))
    onetime_tax_hourly_mix = int(np.round(tax_hourly_fsv_to_start_onetime*100/tax_fsv_to_start_onetime))
    
    
    recurring_cfo_fixed_mix = int(np.round(cfo_fixed_fsv_to_start_recurring*100/cfo_fsv_to_start_recurring))
    recurring_cfo_hourly_mix = int(np.round(cfo_hourly_fsv_to_start_recurring*100/cfo_fsv_to_start_recurring))
    recurring_cpa_fixed_mix = int(np.round(cpa_fixed_fsv_to_start_recurring*100/cpa_fsv_to_start_recurring))
    recurring_cpa_hourly_mix = int(np.round(cpa_hourly_fsv_to_start_recurring*100/cpa_fsv_to_start_recurring))
    recurring_fpa_fixed_mix = int(np.round(fpa_fixed_fsv_to_start_recurring*100/fpa_fsv_to_start_recurring))
    recurring_fpa_hourly_mix = int(np.round(fpa_hourly_fsv_to_start_recurring*100/fpa_fsv_to_start_recurring))
    recurring_bkp_fixed_mix = int(np.round(bkp_fixed_fsv_to_start_recurring*100/bkp_fsv_to_start_recurring))
    recurring_bkp_hourly_mix = int(np.round(bkp_hourly_fsv_to_start_recurring*100/bkp_fsv_to_start_recurring))
    recurring_tax_fixed_mix = int(np.round(tax_fixed_fsv_to_start_recurring*100/tax_fsv_to_start_recurring))
    recurring_tax_hourly_mix = int(np.round(tax_hourly_fsv_to_start_recurring*100/tax_fsv_to_start_recurring))
    
    cfo_onetime_mix = int(np.round(cfo_fsv_to_start_onetime*100/ot_fsv_to_start))
    cpa_onetime_mix = int(np.round(cpa_fsv_to_start_onetime*100/ot_fsv_to_start))
    fpa_onetime_mix = int(np.round(fpa_fsv_to_start_onetime*100/ot_fsv_to_start))
    bkp_onetime_mix = int(np.round(bkp_fsv_to_start_onetime*100/ot_fsv_to_start))
    tax_onetime_mix = int(np.round(tax_fsv_to_start_onetime*100/ot_fsv_to_start))
    
    
    cfo_recurring_mix = int(np.round(cfo_fsv_to_start_recurring*100/rec_fsv_to_start))
    cpa_recurring_mix = int(np.round(cpa_fsv_to_start_recurring*100/rec_fsv_to_start))
    fpa_recurring_mix = int(np.round(fpa_fsv_to_start_recurring*100/rec_fsv_to_start))
    bkp_recurring_mix = int(np.round(bkp_fsv_to_start_recurring*100/rec_fsv_to_start))
    tax_recurring_mix = int(np.round(tax_fsv_to_start_recurring*100/rec_fsv_to_start))
    
    cfo_of_onetime = int(np.round(cfo_fsv_to_start_onetime*100/cfo_fsv_to_start))
    cfo_of_recurring = int(np.round(cfo_fsv_to_start_recurring*100/cfo_fsv_to_start))
    cpa_of_onetime = int(np.round(cpa_fsv_to_start_onetime*100/cpa_fsv_to_start))
    cpa_of_recurring = int(np.round(cpa_fsv_to_start_recurring*100/cpa_fsv_to_start))
    fpa_of_onetime = int(np.round(fpa_fsv_to_start_onetime*100/fpa_fsv_to_start))
    fpa_of_recurring = int(np.round(fpa_fsv_to_start_recurring*100/fpa_fsv_to_start))
    bkp_of_onetime = int(np.round(bkp_fsv_to_start_onetime*100/bkp_fsv_to_start))
    bkp_of_recurring = int(np.round(bkp_fsv_to_start_recurring*100/bkp_fsv_to_start))
    tax_of_onetime = int(np.round(tax_fsv_to_start_onetime*100/tax_fsv_to_start))
    tax_of_recurring = int(np.round(tax_fsv_to_start_recurring*100/tax_fsv_to_start))
    
    cfo_fixed_mix = int(np.round(cfo_fixed_fsv_to_start*100/cfo_fsv_to_start))
    cfo_hourly_mix = int(np.round(cfo_hourly_fsv_to_start*100/cfo_fsv_to_start))
    cpa_fixed_mix = int(np.round(cpa_fixed_fsv_to_start*100/cpa_fsv_to_start))
    cpa_hourly_mix = int(np.round(cpa_hourly_fsv_to_start*100/cpa_fsv_to_start))
    fpa_fixed_mix = int(np.round(fpa_fixed_fsv_to_start*100/fpa_fsv_to_start))
    fpa_hourly_mix = int(np.round(fpa_hourly_fsv_to_start*100/fpa_fsv_to_start))
    bkp_fixed_mix = int(np.round(bkp_fixed_fsv_to_start*100/bkp_fsv_to_start))
    bkp_hourly_mix = int(np.round(bkp_hourly_fsv_to_start*100/bkp_fsv_to_start))
    tax_fixed_mix = int(np.round(tax_fixed_fsv_to_start*100/tax_fsv_to_start))
    tax_hourly_mix = int(np.round(tax_hourly_fsv_to_start*100/tax_fsv_to_start))
    
    
    cfo_fixed_mix = int(np.round(cfo_fixed_fsv_to_start*100/cfo_fsv_to_start))
    cfo_hourly_mix = int(np.round(cfo_hourly_fsv_to_start*100/cfo_fsv_to_start))
    
    fixed_mix = int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))
    hourly_mix = int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))
    
    annual_revenue_0_1_mix = int(np.round(annual_revenue_0_1_increment*100/total_anual_revenue_increment))
    annual_revenue_1_10_mix = int(np.round(annual_revenue_1_10_increment*100/total_anual_revenue_increment))
    annual_revenue_10_50_mix = int(np.round(annual_revenue_10_50_increment*100/total_anual_revenue_increment))
    annual_revenue_5_plus_mix = int(np.round(annual_revenue_5_plus_increment*100/total_anual_revenue_increment))
    annual_revenue_pre_increment_mix = int(np.round(annual_revenue_pre_increment*100/total_anual_revenue_increment))
    annual_revenue_revenue_no_increment_mix = int(np.round(annual_revenue_revenue_no_increment*100/total_anual_revenue_increment))
    
    
    
    
    
#     print(f'OT Mix: {int(np.round(ot_fsv_to_start*100/total_fsv_to_start))}')
#     print(f'Rec Mix: {int(np.round(rec_fsv_to_start*100/total_fsv_to_start))}')
#     print('--------------------------------------------------------------')
#     print(f'CFO Mix: {int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))}, CFO FSV: {cfo_fsv_to_start}')
#     print(f'CPA Mix: {int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))}')
#     print(f'FPA Mix: {int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))}')
#     print(f'BKP Mix: {int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))}')
#     print(f'TAX Mix: {int(np.round(tax_fsv_to_start*100/total_fsv_to_start))}')
#     print('--------------------------------------------------------------')

#     print(f'Fixed Mix: {int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))}')
#     print(f'Hourly Mix: {int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))}')
    
    final_list = [x_0_date, fsv_target, total_fsv_signed, ot_fsv_signed, rec_fsv_signed, total_fsv_launched, new_ot_fsv, new_rec_fsv, carryover_ot_ir, carryover_rec_ir,
                  new_ot_ir, new_rec_ir, ot_mix, rec_mix, cfo_mix, cpa_mix, fpa_mix, bkp_mix, tax_mix, fixed_mix, hourly_mix,
                  onetime_cfo_fixed_mix,onetime_cfo_hourly_mix,onetime_cpa_fixed_mix,onetime_cpa_hourly_mix,onetime_fpa_fixed_mix,onetime_fpa_hourly_mix,onetime_bkp_fixed_mix,onetime_bkp_hourly_mix,onetime_tax_fixed_mix,onetime_tax_hourly_mix,
                 recurring_cfo_fixed_mix,recurring_cfo_hourly_mix,recurring_cpa_fixed_mix,recurring_cpa_hourly_mix,recurring_fpa_fixed_mix,recurring_fpa_hourly_mix,recurring_bkp_fixed_mix,recurring_bkp_hourly_mix,recurring_tax_fixed_mix,recurring_tax_hourly_mix,
                 cfo_onetime_mix,cpa_onetime_mix,fpa_onetime_mix,bkp_onetime_mix,tax_onetime_mix,
                 cfo_recurring_mix,cpa_recurring_mix,fpa_recurring_mix,bkp_recurring_mix,tax_recurring_mix,
                 cfo_of_onetime,cfo_of_recurring,cpa_of_onetime,cpa_of_recurring,fpa_of_onetime,fpa_of_recurring,bkp_of_onetime,
                 bkp_of_recurring,tax_of_onetime,tax_of_recurring,
                 cfo_fixed_mix,cfo_hourly_mix,cpa_fixed_mix,cpa_hourly_mix,fpa_fixed_mix,fpa_hourly_mix,
                 bkp_fixed_mix,bkp_hourly_mix,tax_fixed_mix,tax_hourly_mix,
                 annual_revenue_0_1_mix,annual_revenue_1_10_mix,annual_revenue_10_50_mix,annual_revenue_5_plus_mix,
                 annual_revenue_pre_increment_mix,annual_revenue_revenue_no_increment_mix]
    
    prev_records.append(month_additions)
    final_result = pd.concat([final_result, pd.Series(final_list, index = final_index_vals)], axis=1)

    output_df['fsv_annualized'] = output_df.apply(lambda x: (x.increment/fsv_ot_start_percent)/current_month if x.frequency == 'one_time' else (x.increment*12)/current_month, axis=1)
    output_df.fsv_annualized.to_clipboard()
    break;

5835000
Optimization for the month of '2022-01-01' is successful


In [35]:
output_df

,Optimized,Original,Month_Orig,increment,type,frequency,sl,rate,fsv_annualized
0,30049.214411,0.000000,0.000000,30049.214411,one_time_cfo_fixed,one_time,cfo,fixed,71545.748598
1,79325.114525,49275.900114,49275.900114,30049.214411,one_time_cfo_hourly,one_time,cfo,hourly,71545.748598
2,30049.214412,0.000000,0.000000,30049.214412,one_time_cpaaccounting_advisory_fixed,one_time,cpa,fixed,71545.748599
3,132627.648557,93253.121950,93253.121950,39374.526607,one_time_cpaaccounting_advisory_hourly,one_time,cpa,hourly,93748.872873
4,2250.000000,1000.000000,1000.000000,1250.000000,one_time_fpa_fixed,one_time,fpa,fixed,2976.190476
5,5049.247156,3239.397725,3239.397725,1809.849431,one_time_fpa_hourly,one_time,fpa,hourly,4309.165312
6,30049.214411,0.000000,0.000000,30049.214411,one_time_full_charge_bookkeeping_fixed,one_time,bkp,fixed,71545.748598
7,60478.755298,27663.218988,27663.218988,32815.536310,one_time_full_charge_bookkeeping_hourly,one_time,bkp,hourly,78132.229310
8,1000.000000,0.000000,0.000000,1000.000000,one_time_tax_preparation_fixed,one_time,tax,fixed,2380.952381
9,6300.274912,4240.219929,4240.219929,2060.054982,one_time_tax_preparation_hourly,one_time,tax,hourly,4904.892815


In [33]:
final_result.to_clipboard()

In [32]:
final_result

,record_date,0,0,0,0,0,0,0,0,0,0,0
x_0_date,NaN,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01
fsv_target,NaN,5835000,5820000,5850000,5700000,5550000,5550000,5550000,5400000,5400000,5250000,5100000
total_fsv_signed,NaN,2.16301e+06,2.16305e+06,2.19029e+06,2.12733e+06,2.09666e+06,2.1283e+06,2.1563e+06,2.1458e+06,2.18923e+06,2.17567e+06,2.17304e+06
ot_fsv_signed,NaN,472635,471458,473905,461873,449743,449609,449713,437464,437468,425321,413100
rec_fsv_signed,NaN,1.69038e+06,1.69159e+06,1.71639e+06,1.66546e+06,1.64692e+06,1.67869e+06,1.70659e+06,1.70834e+06,1.75177e+06,1.75034e+06,1.75994e+06
total_fsv_launched,NaN,3.89133e+06,4.24843e+06,4.24466e+06,4.20323e+06,3.99428e+06,3.83009e+06,3.70463e+06,3.43108e+06,3.20532e+06,2.83751e+06,2.47954e+06
new_ot_fsv,NaN,198507,198012,199040,193987,188892,188836,188880,183735,183736,178635,173502
new_rec_fsv,NaN,84518.8,84579.6,85819.4,83273,82346.1,83934.6,85329.4,85416.9,87588.3,87517.2,87997.1
carryover_ot_ir,NaN,0,371452,521551,654343,859482,1.01967e+06,1.15238e+06,1.34494e+06,1.55747e+06,NaN,NaN
carryover_rec_ir,NaN,0,343457,602971,847162,1.02393e+06,1.12911e+06,1.24019e+06,1.22997e+06,1.26247e+06,NaN,NaN


In [265]:
x_0

[389.27083333333337,
 13520.388837345206,
 2919.792361886595,
 179167.08372868676,
 3998.1428009277024,
 3682.3737660733595,
 2919.792262455349,
 21400.18685833149,
 14612.447292375246,
 1111399.406511985,
 310187.37084828666,
 211698.01228267778,
 278260.5003966021,
 188004.3960143992,
 19250.876769469953,
 167024.34907546305,
 311575.85218089586,
 144580.79918768303,
 3464.621769177385,
 18771.336177111116,
 227.9928706666667,
 155.92571733333327,
 167.6418166666667,
 73.09017999999996,
 121.81696666666663,
 538621.3986244948,
 363362.7369689606,
 879424.8205977619,
 267511.14352467796,
 560301.8299268599,
 684.5374180555555]

In [264]:
for i in sol.x:
    print(int(i))

1412
33821
28887
226669
30012
29852
28887
50448
37141
999457
326763
224156
282049
191296
19304
167485
328750
156642
3474
18446
228
161
168
75
126
533825
362978
905376
271007
561243
686


In [299]:
sol

     fun: -3.810575388356575
     jac: array([-5.39466863e-07, -5.28520069e-07, -6.08088832e-07, -5.96738755e-07,
        9.80145932e-07,  3.78143468e-06, -6.36280043e-07, -6.67564580e-07,
       -5.62099808e-07, -5.70332506e-07, -3.26369539e-06, -2.86709251e-06,
       -2.24599664e-06, -1.05074322e-06, -1.92495567e-06, -3.06296421e-06,
       -3.24149515e-06, -1.89419027e-06, -5.62331117e-06,  1.74735320e-06,
        4.77778548e-04, -2.86020470e-03,  4.62758437e-03, -2.35987288e-02,
       -3.98867680e-03, -1.28999190e-07,  6.23007227e-07, -6.65713227e-08,
       -9.99618730e-07,  0.00000000e+00,  3.05381832e-04])
 message: 'Positive directional derivative for linesearch'
    nfev: 2992
     nit: 90
    njev: 86
  status: 8
 success: False
       x: array([3.13171175e+04, 8.30558808e+04, 6.20652697e-12, 1.81326117e+05,
       1.07000000e+03, 3.46615557e+03, 3.62634478e-13, 3.14551502e+04,
       2.81847290e-12, 3.39798312e+05, 2.65232573e+05, 2.00336948e+05,
       2.69891450e+05, 1.8

In [263]:
sol

     fun: -4.972668393778858
     jac: array([-6.70939285e-07, -4.87349787e-07, -6.61850468e-07, -5.81527846e-07,
       -3.46245580e-07, -1.93180638e-07, -5.48879393e-07, -6.02450790e-07,
       -4.77239526e-07, -5.55357198e-07, -3.87516953e-06, -3.45629355e-06,
       -2.77064210e-06, -1.34281317e-06, -2.40393751e-06, -3.74072465e-06,
       -3.98127528e-06, -2.36486375e-06, -6.82197672e-06,  1.80261218e-06,
        7.92230914e-04, -4.06957838e-03,  5.55916717e-03, -2.68633907e-02,
       -3.37165406e-03, -1.38931762e-07,  5.57687178e-07, -5.00201782e-08,
       -1.09227141e-06,  0.00000000e+00,  3.10243439e-04])
 message: 'Positive directional derivative for linesearch'
    nfev: 3683
     nit: 106
    njev: 102
  status: 8
 success: False
       x: array([1.41291667e+03, 3.38218513e+04, 2.88877378e+04, 2.26669645e+05,
       3.00121103e+04, 2.98525319e+04, 2.88877368e+04, 5.04480642e+04,
       3.71413750e+04, 9.99457625e+05, 3.26763299e+05, 2.24156446e+05,
       2.82049778e+05, 1

In [230]:
bounds

((1.0, 100.0),
 (51740.6951195833, 54303.49012527775),
 (1.0, 10000000),
 (186411.02725028663, 10000000),
 (1101.0, 10000000),
 (3564.337497500952, 10000000),
 (1.0, 10000000),
 (32338.07029848704, 10000000),
 (1.0, 10000000),
 (349326.3675906122, 10000000),
 (268780.0, 281678.0),
 (200640.215, 210293.46333333332),
 (277459.5, 10000000),
 (192867.48333333337, 10000000),
 (18948.5, 10000000),
 (164393.25, 10000000),
 (336386.5, 10000000),
 (155527.80000000002, 10000000),
 (3411.0000000000005, 10000000),
 (17673.600000000002, 10000000),
 (214.20000000000002, 234.6),
 (134.4, 147.2),
 (157.5, 172.5),
 (63.0, 69.0),
 (105.0, 114.99999999999999),
 (445330.3844409326, 10000000),
 (262999.71658748324, 10000000),
 (772646.4524641656, 10000000),
 (260897.30226201948, 10000000),
 (537850.2836742304, 10000000),
 (674.75, 10000000))

In [231]:
for i in sol.x:
    print(int(i))

1
51740
1
186411
1101
3564
1
32338
1
349326
268780
200640
277459
192867
18948
164393
336386
155527
3411
17673
214
134
157
63
105
445330
262999
772646
260897
537850
674


In [270]:
output_df

,Optimized,Original,Month_Orig,increment,type,frequency,sl,rate,fsv_annualized
0,1489.037865,1294.815535,1000.000000,194.222330,one_time_cfo_fixed,one_time,cfo,fixed,462.434120
1,3685.685123,3204.943585,2435.714286,480.741538,one_time_cfo_hourly,one_time,cfo,hourly,1144.622709
2,75202.872755,44618.599038,1000.000000,30584.273717,one_time_cpaaccounting_advisory_fixed,one_time,cpa,fixed,72819.699326
3,75315.727381,44726.080370,1079.715596,30589.647011,one_time_cpaaccounting_advisory_hourly,one_time,cpa,hourly,72832.492883
4,75202.866295,44618.595319,1000.000000,30584.270976,one_time_fpa_fixed,one_time,fpa,fixed,72819.692800
5,74184.692112,43648.906446,101.800554,30535.785666,one_time_fpa_hourly,one_time,fpa,hourly,72704.251585
6,75202.869099,44618.596826,1000.000000,30584.272273,one_time_full_charge_bookkeeping_fixed,one_time,bkp,fixed,72819.695889
7,74099.403570,43567.676920,26.923077,30531.726651,one_time_full_charge_bookkeeping_hourly,one_time,bkp,hourly,72694.587263
8,75194.946584,44611.051968,1000.000000,30583.894616,one_time_tax_preparation_fixed,one_time,tax,fixed,72818.796704
9,285017.884956,244442.421214,184948.246307,40575.463742,one_time_tax_preparation_hourly,one_time,tax,hourly,96608.247006


In [209]:
temp = x_0_transform.reset_index()
temp.loc[temp['entry_month']=='2022-01-01']

,entry_month,one_time_cfo_fixed,one_time_cfo_hourly,one_time_cpaaccounting_advisory_fixed,one_time_cpaaccounting_advisory_hourly,one_time_fpa_fixed,one_time_fpa_hourly,one_time_full_charge_bookkeeping_fixed,one_time_full_charge_bookkeeping_hourly,one_time_tax_preparation_fixed,one_time_tax_preparation_hourly,recurring_monthly_cfo_fixed,recurring_monthly_cfo_hourly,recurring_monthly_cpaaccounting_advisory_fixed,recurring_monthly_cpaaccounting_advisory_hourly,recurring_monthly_fpa_fixed,recurring_monthly_fpa_hourly,recurring_monthly_full_charge_bookkeeping_fixed,recurring_monthly_full_charge_bookkeeping_hourly,recurring_monthly_tax_preparation_fixed,recurring_monthly_tax_preparation_hourly,cfo_price,cpa_price,fpa_price,book_price,tax_price,revenue_0_1,revenue_1_10,revenue_10_50,revenue_5_plus,revenue_pre,revenue_no
60,2022-01-01,0.0,49275.900114,0.0,169463.661137,1000.0,3239.397725,0.0,29397.336635,0.0,317568.515991,255980.0,191084.966667,252235.0,175333.166667,17225.0,149447.5,305805.0,141388.0,3100.0,16066.0,204.0,128.0,150.0,60.0,100.0,404844.894946,239089.742352,702404.956786,237178.456602,488953.894249,612.5


In [197]:
list(output_df.Optimized[Mix.cfo.value])
output_df.loc[(output_df['frequency']=='one_time') & (output_df['rate']=='fixed') & (output_df['sl']=='cfo')]['Original'].sum()

348584.70657979534

In [277]:
output_df

,Optimized,Original,Month_Orig,increment,type,frequency,sl,rate,fsv_annualized
0,1489.037865,1294.815535,1000.000000,194.222330,one_time_cfo_fixed,one_time,cfo,fixed,462.434120
1,3685.685123,3204.943585,2435.714286,480.741538,one_time_cfo_hourly,one_time,cfo,hourly,1144.622709
2,75202.872755,44618.599038,1000.000000,30584.273717,one_time_cpaaccounting_advisory_fixed,one_time,cpa,fixed,72819.699326
3,75315.727381,44726.080370,1079.715596,30589.647011,one_time_cpaaccounting_advisory_hourly,one_time,cpa,hourly,72832.492883
4,75202.866295,44618.595319,1000.000000,30584.270976,one_time_fpa_fixed,one_time,fpa,fixed,72819.692800
5,74184.692112,43648.906446,101.800554,30535.785666,one_time_fpa_hourly,one_time,fpa,hourly,72704.251585
6,75202.869099,44618.596826,1000.000000,30584.272273,one_time_full_charge_bookkeeping_fixed,one_time,bkp,fixed,72819.695889
7,74099.403570,43567.676920,26.923077,30531.726651,one_time_full_charge_bookkeeping_hourly,one_time,bkp,hourly,72694.587263
8,75194.946584,44611.051968,1000.000000,30583.894616,one_time_tax_preparation_fixed,one_time,tax,fixed,72818.796704
9,285017.884956,244442.421214,184948.246307,40575.463742,one_time_tax_preparation_hourly,one_time,tax,hourly,96608.247006


In [160]:
x_0

[9623.611111111108,
 29020.739305917603,
 102361.29940050472,
 178937.083145572,
 3768.1422047485667,
 3452.373159766002,
 2689.791677363598,
 56497.71615709993,
 2689.791599732976,
 796525.3387712374,
 328237.3108879937,
 211380.55851310404,
 278260.5004100425,
 188004.39602053098,
 19250.87678094723,
 167024.34909389887,
 311575.85220042313,
 144580.79919897212,
 3464.6218032684924,
 18771.336177111116,
 227.9928706666667,
 155.92571733333332,
 167.64181666666667,
 73.09018000000007,
 121.81696666666666,
 538621.398624885,
 363362.7369689606,
 879424.8205988595,
 267511.14353281364,
 560301.8299268599,
 684.5374180555555]

In [161]:
bounds

((10104.791666666664, 16548.33333333333),
 (30471.776271213483, 39824.88716710112),
 (107479.36437052996, 10000000),
 (187883.9373028506, 10000000),
 (3956.5493149859954, 10000000),
 (3624.9918177543022, 10000000),
 (2824.281261231778, 10000000),
 (59322.60196495493, 10000000),
 (2824.281179719625, 10000000),
 (836351.6057097993, 10000000),
 (344649.17643239343, 443884.77306559245),
 (221949.58643875926, 303656.67021572485),
 (292173.5254305447, 10000000),
 (197404.61582155756, 10000000),
 (20213.42061999459, 10000000),
 (175375.5665485938, 10000000),
 (327154.6448104443, 10000000),
 (151809.83915892075, 10000000),
 (3637.852893431917, 10000000),
 (19709.902985966673, 10000000),
 (239.39251420000005, 262.1918012666667),
 (163.7220032, 179.31457493333332),
 (176.0239075, 192.78808916666665),
 (76.74468900000008, 84.05370700000007),
 (127.907815, 140.08951166666665),
 (565552.4685561293, 10000000),
 (381530.87381740863, 10000000),
 (923396.0616288024, 10000000),
 (280886.70070945437, 100

In [162]:
for i in sol.x:
    print(int(i))

14699
54935
96968
224394
27736
27576
26612
73109
26612
775080
339407
223710
282049
191296
19304
167485
328750
156642
3474
18446
228
161
168
75
126
533825
362978
905376
271007
561243
686


In [38]:
output_df['Original']

0          0.000000
1      49275.900114
2          0.000000
3     169463.661137
4       1000.000000
5       3239.397725
6          0.000000
7      29397.336635
8          0.000000
9     317568.515991
10    255980.000000
11    191084.966667
12    252235.000000
13    175333.166667
14     17225.000000
15    149447.500000
16    305805.000000
17    141388.000000
18      3100.000000
19     16066.000000
20       204.000000
21       128.000000
22       150.000000
23        60.000000
24       100.000000
25    404844.894946
26    239089.742352
27    702404.956786
28    237178.456602
29    488953.894249
30       612.500000
Name: Original, dtype: float64

In [39]:
output_df['Optimized']

0     1.000000e+02
1     5.175085e+04
2     1.873234e-18
3     1.779368e+05
4     1.050000e+03
5     3.401368e+03
6     4.144915e-18
7     3.086720e+04
8     2.423069e-19
9     3.334469e+05
10    2.687790e+05
11    2.006392e+05
12    2.648468e+05
13    1.840998e+05
14    1.808625e+04
15    1.569199e+05
16    3.210952e+05
17    1.484574e+05
18    3.255000e+03
19    1.686930e+04
20    2.040000e+02
21    1.280000e+02
22    1.500000e+02
23    6.000000e+01
24    1.000000e+02
25    4.250871e+05
26    2.510442e+05
27    7.375252e+05
28    2.490374e+05
29    5.134016e+05
30    6.431250e+02
Name: Optimized, dtype: float64

In [178]:
Mix.cfo.value

[0, 1, 10, 11]

In [154]:
objective_fn_forecast(sol.x)

(1193142.7042857108, 2360992.5076105655)

In [139]:
objective_fn_forecast(x_0)

(1315569.064691596, 2150783.5167676248)

In [171]:
output_df.Optimized[Mix.cfo.value]

0      61692.538948
1      63337.755117
10    234791.743615
11    227622.840002
Name: Optimized, dtype: float64

In [290]:
output_df

,Optimized,Original,Month_Orig,increment,type,frequency,sl,rate,fsv_annualized
0,18588.827580,0.000000,0.000000,18588.827580,one_time_cfo_fixed,one_time,cfo,fixed,44259.113287
1,70328.522632,49275.900114,49275.900114,21052.622518,one_time_cfo_hourly,one_time,cfo,hourly,50125.291711
2,15415.592636,0.000000,0.000000,15415.592636,one_time_cpaaccounting_advisory_fixed,one_time,cpa,fixed,36703.791991
3,193352.436254,169463.661137,169463.661137,23888.775117,one_time_cpaaccounting_advisory_hourly,one_time,cpa,hourly,56878.035993
4,16465.590564,1000.000000,1000.000000,15465.590564,one_time_fpa_fixed,one_time,fpa,fixed,36822.834677
5,18816.957131,3239.397725,3239.397725,15577.559406,one_time_fpa_hourly,one_time,fpa,hourly,37089.427157
6,15415.591819,0.000000,0.000000,15415.591819,one_time_full_charge_bookkeeping_fixed,one_time,bkp,fixed,36703.790046
7,46282.795626,29397.336635,29397.336635,16885.458991,one_time_full_charge_bookkeeping_hourly,one_time,bkp,hourly,40203.473788
8,15415.591222,0.000000,0.000000,15415.591222,one_time_tax_preparation_fixed,one_time,tax,fixed,36703.788624
9,348862.533770,317568.515991,317568.515991,31294.017779,one_time_tax_preparation_hourly,one_time,tax,hourly,74509.566139


In [279]:
# sol

In [289]:
final_result

,record_date,0
x_0_date,NaN,2022-01-01
fsv_target,NaN,5.55518e+06
total_fsv_signed,NaN,2.66622e+06
ot_fsv_signed,NaN,449999
rec_fsv_signed,NaN,2.21622e+06
total_fsv_launched,NaN,4.8737e+06
new_ot_fsv,NaN,189000
new_rec_fsv,NaN,110811
carryover_ot_ir,NaN,0
carryover_rec_ir,NaN,0


In [175]:
# sol

In [206]:
final_result.to_clipboard()

In [ ]:
final_result

In [81]:
sol

     fun: -7.958712558576825
     jac: array([-6.99756034e-07, -7.60043314e-07, -8.99061178e-07, -9.40306716e-07,
       -6.33236933e-07, -5.85519308e-07, -7.70116284e-07, -7.08184617e-07,
       -9.17130793e-07, -3.95151464e-07, -8.15211778e-06, -7.35513887e-06,
       -8.17201308e-06, -7.44924443e-06, -7.53279618e-06, -6.41738135e-06,
       -9.46699527e-06, -7.21091289e-06, -1.41553453e-05, -1.28235746e-05])
 message: 'Positive directional derivative for linesearch'
    nfev: 941
     nit: 205
    njev: 41
  status: 8
 success: False
       x: array([ 49529.72235996,  39625.11190092,  49486.2815514 ,  36487.45593837,
        49367.26307423,  32421.90751751,  49349.2680688 ,  31672.63464202,
        49368.01483917,  54944.38908626, 552043.14761687, 326983.80778047,
       516673.28276358, 352379.13298403,  66126.09854773, 129446.95973893,
       689400.87632705, 400214.20233533,  30399.93500739,  85802.98403837])

## June

In [52]:
## Constraints

x_0_date = '2021-06-01'
x_0 = x_0_transform.filter(like = x_0_date, axis=0).values[0].tolist()

fsv_target = 3984375 #June
# fsv_previous_month = 3362500
# fsv_2_months_back = 3500000
# fsv_3_months_back = 3000000

one_time_mix_lower = 0.09
one_time_mix_upper = 0.15

recurring_mix_lower = 0.5
recurring_mix_upper = 0.8

cfo_mix_lower = 0.25
cfo_mix_upper = 0.35

cpa_mix_lower = 0.21
cpa_mix_upper = 0.32

fpa_mix_lower = 0.03
fpa_mix_upper = 0.1

bkp_mix_lower = 0.5
bkp_mix_upper = 0.26

tax_mix_lower = 0.05
tax_mix_upper = 0.08

fixed_mix_lower = 0.3
fixed_mix_upper = 0.7

hourly_mix_lower = 0.15
hourly_mix_upper = 0.4

#Realization ratios
current_month = 0.5
previous_month = 0.25
two_months_back = 0.1
three_months_back = 0.05

#
churn_rate = 0.9

fsv_ot_start_percent = 0.7

target_achieve_factor = 0.9
target_miss_factor = 0.8


fsv_to_start = current_month*fsv_target #+ previous_month*fsv_previous_month + two_months_back*fsv_2_months_back + three_months_back*fsv_3_months_back
fsv_to_start = fsv_to_start*target_achieve_factor

upper_limit_one_time_pr_lower = fsv_to_start*one_time_mix_lower


def total_fsv_constraint(x):
    current_fsv = sum(x_0)
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    #additional_fsv = fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12
    
    #return (current_fsv + additional_fsv) - sum(x)
    return fsv_to_start - (ot_annual_fsv + rec_annual_fsv)

def total_fsv_constraint_lower(x):
    current_fsv = sum(x_0)
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
#     additional_fsv = fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12
    
#     return ot_annual_fsv + rec_annual_fsvsum(x) - target_miss_factor*(current_fsv + additional_fsv)
    return (ot_annual_fsv + rec_annual_fsv) - target_miss_factor*fsv_to_start

def one_time_fsv_constraint(x):
    current_fsv = sum(x_0[0:10])
    additional_fsv = fsv_to_start*one_time_mix_upper*fsv_ot_start_percent 
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (fsv_to_start)*one_time_mix_upper - ot_annual_fsv
#     return (current_fsv + additional_fsv) - sum(x[0:10])

def one_time_fsv_constraint_lower(x):
    current_fsv = sum(x_0[0:10])
    additional_fsv = fsv_to_start*one_time_mix_upper*fsv_ot_start_percent 
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return ot_annual_fsv - (fsv_to_start)*one_time_mix_lower 
#     return (current_fsv + additional_fsv) - sum(x[0:10])

def hourly_fsv_constraint(x):
    
    current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
    additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper
    
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return fsv_to_start*hourly_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def hourly_fsv_constraint_lower(x):
    
    current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
    additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper
    
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*hourly_mix_lower

def cfo_fsv_constraint(x):
#     current_fsv = x_0[Mix.one_time_cfo_fixed.value] + x_0[Mix.one_time_cfo_hourly.value] + x_0[Mix.recurring_monthly_cfo_fixed.value] + x_0[Mix.recurring_monthly_cfo_hourly.value]
#     additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*cfo_mix_upper
    
#     return (current_fsv + additional_fsv) - (x[Mix.one_time_cfo_fixed.value] + x[Mix.one_time_cfo_hourly.value] + 
#                                              x[Mix.recurring_monthly_cfo_fixed.value] + x[Mix.recurring_monthly_cfo_hourly.value])

    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*cfo_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def cfo_fsv_constraint_lower(x):
#     current_fsv = x_0[Mix.one_time_cfo_fixed.value] + x_0[Mix.one_time_cfo_hourly.value] + x_0[Mix.recurring_monthly_cfo_fixed.value] + x_0[Mix.recurring_monthly_cfo_hourly.value]
#     additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*cfo_mix_upper
    
#     return (current_fsv + additional_fsv) - (x[Mix.one_time_cfo_fixed.value] + x[Mix.one_time_cfo_hourly.value] + 
#                                              x[Mix.recurring_monthly_cfo_fixed.value] + x[Mix.recurring_monthly_cfo_hourly.value])

    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*cfo_mix_lower

def fpa_fsv_constraint(x):
#     current_fsv = x_0[Mix.one_time_fpa_fixed.value] + x_0[Mix.one_time_fpa_hourly.value] + x_0[Mix.recurring_monthly_fpa_fixed.value] + x_0[Mix.recurring_monthly_fpa_hourly.value]
#     additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*fpa_mix_upper
    
#     return (current_fsv + additional_fsv) - (x[Mix.one_time_fpa_fixed.value] + x[Mix.one_time_fpa_hourly.value] + 
#                                              x[Mix.recurring_monthly_fpa_fixed.value] + x[Mix.recurring_monthly_fpa_hourly.value])

    ot_addition = sum(np.array(x)[Mix.fpa.value][0:2]) - sum(np.array(x_0)[Mix.fpa.value][0:2])
    rec_addition = sum(np.array(x)[Mix.fpa.value][2:]) - sum(np.array(x_0)[Mix.fpa.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*fpa_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def tax_fsv_constraint(x):
#     current_fsv = x_0[Mix.one_time_tax_preparation_fixed.value] + x_0[Mix.one_time_tax_preparation_hourly.value] + x_0[Mix.recurring_monthly_tax_preparation_fixed.value] + x_0[Mix.recurring_monthly_tax_preparation_hourly.value]
#     additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*tax_mix_upper
    
#     return (current_fsv + additional_fsv) - (x[Mix.one_time_tax_preparation_fixed.value] + x[Mix.one_time_tax_preparation_hourly.value] + 
#                                              x[Mix.recurring_monthly_tax_preparation_fixed.value] + x[Mix.recurring_monthly_tax_preparation_hourly.value])
    ot_addition = sum(np.array(x)[Mix.tax.value][0:2]) - sum(np.array(x_0)[Mix.tax.value][0:2])
    rec_addition = sum(np.array(x)[Mix.tax.value][2:]) - sum(np.array(x_0)[Mix.tax.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*tax_mix_upper - (ot_annual_fsv + rec_annual_fsv)  
    
# def one_time_pr_lower_constraint(x):
    
#     ot_pr = sum(x[0:10])
#     current_ot_pr = sum(x_0[0:10])
    
#     return ot_pr - current_ot_pr - fsv_to_start * one_time_mix_lower

# def one_time_pr_upper_constraint(x):
    
#     ot_pr = sum(x[0:10])
#     current_ot_pr = sum(x_0[0:10])
    
#     return fsv_to_start*one_time_mix_upper - ot_pr - current_ot_pr 

# def recurring_pr_lower_constraint(x):
    
#     rec_pr = sum(x[10:])
#     current_rec_pr = sum(x_0[10:])
    
#     return rec_pr - current_rec_pr - fsv_to_start * recurring_mix_lower/12

# def recurring_pr_upper_constraint(x):
    
#     rec_pr = sum(x[10:])
#     current_rec_pr = sum(x_0[10:])
    
#     return fsv_to_start*recurring_mix_upper/12 - rec_pr - current_rec_pr

total_fsv_cons = {'type':'ineq', 'fun':total_fsv_constraint}
total_fsv_cons_lower = {'type':'ineq', 'fun':total_fsv_constraint_lower}
ot_fsv_cons = {'type':'ineq', 'fun':one_time_fsv_constraint}
ot_fsv_cons_lower = {'type':'ineq', 'fun':one_time_fsv_constraint_lower}

hourly_fsv_cons = {'type':'ineq', 'fun':hourly_fsv_constraint}
hourly_fsv_cons_lower = {'type':'ineq', 'fun':hourly_fsv_constraint_lower}
cfo_fsv_cons = {'type':'ineq', 'fun':cfo_fsv_constraint}
cfo_fsv_cons_lower = {'type':'ineq', 'fun':cfo_fsv_constraint_lower}
fpa_fsv_cons = {'type':'ineq', 'fun':fpa_fsv_constraint}
tax_fsv_cons = {'type':'ineq', 'fun':tax_fsv_constraint}



# ot_pr_lower_cons = {'type':'ineq', 'fun':one_time_pr_lower_constraint}
# ot_pr_upper_cons = {'type':'ineq', 'fun':one_time_pr_upper_constraint}
# rec_pr_lower_cons = {'type':'ineq', 'fun':recurring_pr_lower_constraint}
# rec_pr_upper_cons = {'type':'ineq', 'fun':recurring_pr_upper_constraint}

constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, ot_fsv_cons_lower, hourly_fsv_cons, hourly_fsv_cons_lower, cfo_fsv_cons, cfo_fsv_cons_lower, fpa_fsv_cons, tax_fsv_cons ]
#constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, hourly_fsv_cons, cfo_fsv_cons, fpa_fsv_cons, tax_fsv_cons]
#constraints = [total_fsv_cons, ot_pr_lower_cons, ot_pr_upper_cons, rec_pr_lower_cons, rec_pr_upper_cons]

bounds = ()
for idx, val in enumerate(x_0):
    bound_val = (x_0[idx] *1.05,500000)
    bounds = bounds + (bound_val,)
    
trend = 41
print_mode = 0
opt_method = 'COBYLA'
# opt_method = 'L-BFGS-B'
opt_method = 'SLSQP'
sol = minimize(objective_fn, x_0, method=opt_method, bounds = bounds, constraints=constraints, options={'maxiter': 500000, 'disp': True ,'eps' : 0.5})

output_df = pd.DataFrame([sol.x, x_0]).T
output_df.columns = ['Optimized', 'Original']
output_df['Optimized'] = output_df.apply(lambda x: x.Optimized if x.Optimized > x.Original else x.Original, axis=1)
output_df['increment'] = output_df['Optimized'] - output_df['Original']
output_df['type'] = output_columns
output_df['frequency'] = output_frequency
output_df['sl'] = output_sl
output_df['rate'] = output_rate_type
june_ot_additions = list(output_df['increment'])[0:10]
june_rec_additions = list(output_df['increment'])[10:]
june_additions = list(output_df['increment'])
print_mode = 0
# print(f'Total FSV signed for the month: {sum(list(output_df.Optimized))}')
# print(f'New FSV for the month: {1.11*(sum(list(output_df.Optimized)) - sum(list(output_df.Original)))}')
# print(f'New IR for the month: {-1.11*1000*(objective_fn(list(output_df.Optimized)) - objective_fn(list(output_df.Original)))}')
# print(f'Total FSV for the month: {1.11*sum(list(output_df.Optimized))}')
print(sol)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1459.0288720606409
            Iterations: 109
            Function evaluations: 2268
            Gradient evaluations: 108
     fun: -1459.0288720606409
     jac: array([-0.00071545, -0.00076847, -0.00083167, -0.00086696, -0.00054262,
       -0.00048862, -0.00077424, -0.00070281, -0.00088436,  0.00792159,
       -0.00111143, -0.00078147, -0.00126378, -0.00092203, -0.00070129,
       -0.00011816, -0.00158881, -0.00100298, -0.00280462, -0.00228156])
 message: 'Optimization terminated successfully'
    nfev: 2268
     nit: 109
    njev: 108
  status: 0
 success: True
       x: array([ 34830.8265494 ,  55435.31099954,  27239.7057204 ,  72646.70726297,
        15744.41387334,  23131.17660194,  31558.85967581,  35884.86276431,
        33406.53076731,   1480.40986889, 178966.00354956, 131808.6       ,
       246777.52857441, 158607.09375   ,  42189.        ,  22701.        ,
       381455.56552191, 223

In [53]:
ot_fsv_to_start = (sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/fsv_ot_start_percent
rec_fsv_to_start = (sum(list(output_df.Optimized)[10:]) - sum(list(output_df.Original)[10:]))*12
total_fsv_to_start = ot_fsv_to_start + rec_fsv_to_start

(ot_opt, rec_opt) = objective_fn_forecast(output_df.Optimized)
(ot, rec) = objective_fn_forecast(output_df.Original)


print(f'Total FSV signed for the month: {total_fsv_to_start/current_month}')
print(f'Total OT FSV signed for the month: {ot_fsv_to_start/current_month}')
print(f'Total Rec FSV signed for the month: {rec_fsv_to_start/current_month}')
print('--------------------------------------------------------------')
print(f'Total FSV launched for the month: {1.07*(sum(list(output_df.Optimized)))}')
# print(f'Total OT FSV launched for the month: {ot_fsv_to_start}')
# print(f'Total Rec FSV launched for the month: {rec_fsv_to_start}')
print('--------------------------------------------------------------')
print(f'New OT FSV applicable for month: {ot_fsv_to_start*fsv_ot_start_percent}')
print(f'New Rec FSV applicable for month: {rec_fsv_to_start/12}')
print('--------------------------------------------------------------')
print(f'New OT IR for month: {ot_opt - ot}')
print(f'New Rec IR applicable for month: {0.65*(rec_opt - rec)}')
print('--------------------------------------------------------------')


cfo_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
cpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
fpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
bkp_fsv_to_start = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
tax_fsv_to_start = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12

fixed_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fixed.value])[0:5]) - sum(list(output_df.Original[Mix.fixed.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fixed.value])[5:]) - sum(list(output_df.Original[Mix.fixed.value])[5:]))*12
hourly_fsv_to_start = ((sum(list(output_df.Optimized[Mix.hourly.value])[0:5]) - sum(list(output_df.Original[Mix.hourly.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.hourly.value])[5:]) - sum(list(output_df.Original[Mix.hourly.value])[5:]))*12


print(f'OT Mix: {int(np.round(ot_fsv_to_start*100/total_fsv_to_start))}')
print(f'Rec Mix: {int(np.round(rec_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')
print(f'CFO Mix: {int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))}, CFO FSV: {cfo_fsv_to_start}')
print(f'CPA Mix: {int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'FPA Mix: {int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'BKP Mix: {int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))}')
print(f'TAX Mix: {int(np.round(tax_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')

print(f'Fixed Mix: {int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))}')
print(f'Hourly Mix: {int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))}')


output_df['fsv_annualized'] = output_df.apply(lambda x: (x.increment/fsv_ot_start_percent)/current_month if x.frequency == 'one_time' else (x.increment*12)/current_month, axis=1)
output_df.fsv_annualized.to_clipboard()


Total FSV signed for the month: 3585937.5
Total OT FSV signed for the month: 537890.625
Total Rec FSV signed for the month: 3048046.875
--------------------------------------------------------------
Total FSV launched for the month: 1882183.373963546
--------------------------------------------------------------
New OT FSV applicable for month: 188261.71875
New Rec FSV applicable for month: 127001.953125
--------------------------------------------------------------
New OT IR for month: 142376.52113446136
New Rec IR applicable for month: 105588.45231440302
--------------------------------------------------------------
OT Mix: 15
Rec Mix: 85
--------------------------------------------------------------
CFO Mix: 26, CFO FSV: 469050.3676646106
CPA Mix: 25
FPA Mix: 4
BKP Mix: 37
TAX Mix: 8
--------------------------------------------------------------
Fixed Mix: 76
Hourly Mix: 24


In [405]:
#output_df

In [271]:
one_time_fsv_constraint_lower(sol.x)
sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]) 


18324.143913243795

In [473]:
(sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/(0.3*0.45)
#(sum(list(output_df.Optimized)[10:]) - sum(list(output_df.Original)[10:]))/(0.45/12)

83668.91381432855/(3307090.5053450353/12)
68717.54372175649/406046.3354657621
current_month
0.875**7

0.39269590377807617

## July

In [54]:
## Constraints

x_0_date = '2021-07-01'
x_0 = x_0_transform.filter(like = x_0_date, axis=0).values[0].tolist()
x_0_orig = x_0
# adding in projects already launched
#
churn_rate = 0.9

x_0 = [(x_0[idx] + june_additions[idx]) if idx <10 else (x_0[idx] + churn_rate*june_additions[idx]) for idx,value in enumerate(x_0) ]

# adding projects not yet launched
x_0 = [(x_0[idx] + june_additions[idx]*previous_month/current_month) for idx,value in enumerate(x_0) ]


fsv_target = 4228125 #July
fsv_previous_month = 4118750 #June
fsv_2_months_back = 0
fsv_3_months_back = 0


one_time_mix_lower = 0.09
one_time_mix_upper = 0.15

recurring_mix_lower = 0.5
recurring_mix_upper = 0.8

cfo_mix_lower = 0.25
cfo_mix_upper = 0.35

cpa_mix_lower = 0.21
cpa_mix_upper = 0.32

fpa_mix_lower = 0.03
fpa_mix_upper = 0.1

bkp_mix_lower = 0.5
bkp_mix_upper = 0.26

tax_mix_lower = 0.05
tax_mix_upper = 0.08

fixed_mix_lower = 0.3
fixed_mix_upper = 0.7

hourly_mix_lower = 0.15
hourly_mix_upper = 0.4

#Realization ratios
current_month = 0.5
previous_month = 0.25
two_months_back = 0.1
three_months_back = 0.05


fsv_ot_start_percent = 0.7

target_achieve_factor = 0.9
target_miss_factor = 0.8


fsv_to_start = current_month*fsv_target #+ previous_month*fsv_previous_month + two_months_back*fsv_2_months_back + three_months_back*fsv_3_months_back
fsv_to_start = fsv_to_start*target_achieve_factor

upper_limit_one_time_pr_lower = fsv_to_start*one_time_mix_lower


def total_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return fsv_to_start - (ot_annual_fsv + rec_annual_fsv)

def total_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (ot_annual_fsv + rec_annual_fsv) - target_miss_factor*fsv_to_start

def one_time_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (fsv_to_start)*one_time_mix_upper - ot_annual_fsv

def one_time_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return ot_annual_fsv - (fsv_to_start)*one_time_mix_lower 

def hourly_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return fsv_to_start*hourly_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def hourly_fsv_constraint_lower(x):
    
    current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
    additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper
    
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*hourly_mix_lower

def cfo_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*cfo_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def cfo_fsv_constraint_lower(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*cfo_mix_lower

def fpa_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.fpa.value][0:2]) - sum(np.array(x_0)[Mix.fpa.value][0:2])
    rec_addition = sum(np.array(x)[Mix.fpa.value][2:]) - sum(np.array(x_0)[Mix.fpa.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*fpa_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def tax_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.tax.value][0:2]) - sum(np.array(x_0)[Mix.tax.value][0:2])
    rec_addition = sum(np.array(x)[Mix.tax.value][2:]) - sum(np.array(x_0)[Mix.tax.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*tax_mix_upper - (ot_annual_fsv + rec_annual_fsv)  
    
total_fsv_cons = {'type':'ineq', 'fun':total_fsv_constraint}
total_fsv_cons_lower = {'type':'ineq', 'fun':total_fsv_constraint_lower}
ot_fsv_cons = {'type':'ineq', 'fun':one_time_fsv_constraint}
ot_fsv_cons_lower = {'type':'ineq', 'fun':one_time_fsv_constraint_lower}

hourly_fsv_cons = {'type':'ineq', 'fun':hourly_fsv_constraint}
hourly_fsv_cons_lower = {'type':'ineq', 'fun':hourly_fsv_constraint_lower}
cfo_fsv_cons = {'type':'ineq', 'fun':cfo_fsv_constraint}
cfo_fsv_cons_lower = {'type':'ineq', 'fun':cfo_fsv_constraint_lower}
fpa_fsv_cons = {'type':'ineq', 'fun':fpa_fsv_constraint}
tax_fsv_cons = {'type':'ineq', 'fun':tax_fsv_constraint}

constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, ot_fsv_cons_lower, hourly_fsv_cons, hourly_fsv_cons_lower, cfo_fsv_cons, cfo_fsv_cons_lower, fpa_fsv_cons, tax_fsv_cons ]

bounds = ()
for idx, val in enumerate(x_0):
    bound_val = (x_0[idx] *1.05,500000)
    bounds = bounds + (bound_val,)
    
trend = 41
print_mode = 0
opt_method = 'COBYLA'
# opt_method = 'L-BFGS-B'
opt_method = 'SLSQP'
sol = minimize(objective_fn, x_0, method=opt_method, bounds = bounds, constraints=constraints, options={'maxiter': 500000, 'disp': True ,'eps' : 0.5})

output_df = pd.DataFrame([sol.x, x_0, x_0_orig]).T
output_df.columns = ['Optimized', 'Original', 'Month_Orig']
output_df['Optimized'] = output_df.apply(lambda x: x.Optimized if x.Optimized > x.Original else x.Original, axis=1)
output_df['increment'] = output_df['Optimized'] - output_df['Original']
output_df['type'] = output_columns
output_df['frequency'] = output_frequency
output_df['sl'] = output_sl
output_df['rate'] = output_rate_type
july_ot_additions = list(1.11*output_df['increment'])[0:10]
july_rec_additions = list(1.11*output_df['increment'])[10:]
july_additions = list(output_df['increment'])
print_mode = 0
print(f'New FSV for the month: {1.11*(sum(list(output_df.Optimized)) - sum(list(output_df.Original)))}')
print(f'New IR for the month: {-1.11*1000*(objective_fn(list(output_df.Optimized)) - objective_fn(list(output_df.Original)))}')
print(f'Total FSV for the month: {1.11*sum(list(output_df.Optimized))}')
print(sol)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1948.4517744347595
            Iterations: 106
            Function evaluations: 2205
            Gradient evaluations: 105
New FSV for the month: 371350.9335937526
New IR for the month: 392313.76381688204
Total FSV for the month: 2459153.7051898493
     fun: -1948.4517744347595
     jac: array([-0.00072003, -0.00075348, -0.00080083, -0.0008194 , -0.00064248,
       -0.00061955, -0.00075308, -0.00071012, -0.00081002,  0.00871415,
       -0.00145376, -0.00113493, -0.00160411, -0.00120984, -0.00093805,
       -0.0002424 , -0.00197462, -0.00129368, -0.00308367, -0.00264797])
 message: 'Optimization terminated successfully'
    nfev: 2205
     nit: 106
    njev: 105
  status: 0
 success: True
       x: array([ 56788.74814641,  84086.57907974,  66262.70335113,  87197.12324659,
        43616.12771414,  44618.3085136 ,  57052.59782272,  54679.54436469,
        60999.27186124,   1339.44060906, 227804.337

In [55]:
ot_fsv_to_start = (sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/fsv_ot_start_percent
rec_fsv_to_start = (sum(list(output_df.Optimized)[10:]) - sum(list(output_df.Original)[10:]))*12
total_fsv_to_start = ot_fsv_to_start + rec_fsv_to_start

(ot_opt, rec_opt) = objective_fn_forecast(output_df.Optimized)
(ot, rec) = objective_fn_forecast(output_df.Month_Orig)
(ot_new, rec_new) = objective_fn_forecast(output_df.Original)


print(f'Total FSV signed for the month: {total_fsv_to_start/current_month}')
print(f'Total OT FSV signed for the month: {ot_fsv_to_start/current_month}')
print(f'Total Rec FSV signed for the month: {rec_fsv_to_start/current_month}')
print('--------------------------------------------------------------')
print(f'Total FSV launched for the month: {1.07*(sum(list(output_df.Optimized)))}')
# print(f'Total OT FSV launched for the month: {ot_fsv_to_start}')
# print(f'Total Rec FSV launched for the month: {rec_fsv_to_start}')
print('--------------------------------------------------------------')
print(f'New OT FSV applicable for month: {ot_fsv_to_start*fsv_ot_start_percent}')
print(f'New Rec FSV applicable for month: {rec_fsv_to_start/12}')
print('--------------------------------------------------------------')
print(f'Carryover OT IR for month: {ot_new - ot}')
print(f'Carryover Rec IR applicable for month: {(rec_new - rec)}')
print('--------------------------------------------------------------')
print(f'New OT IR for month: {ot_opt - ot_new}')
print(f'New Rec IR applicable for month: {0.525*(rec_opt - rec_new)}')
print('--------------------------------------------------------------')


cfo_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
cpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
fpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
bkp_fsv_to_start = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
tax_fsv_to_start = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12

fixed_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fixed.value])[0:5]) - sum(list(output_df.Original[Mix.fixed.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fixed.value])[5:]) - sum(list(output_df.Original[Mix.fixed.value])[5:]))*12
hourly_fsv_to_start = ((sum(list(output_df.Optimized[Mix.hourly.value])[0:5]) - sum(list(output_df.Original[Mix.hourly.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.hourly.value])[5:]) - sum(list(output_df.Original[Mix.hourly.value])[5:]))*12


print(f'OT Mix: {int(np.round(ot_fsv_to_start*100/total_fsv_to_start))}')
print(f'Rec Mix: {int(np.round(rec_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')
print(f'CFO Mix: {int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))}, CFO FSV: {cfo_fsv_to_start}')
print(f'CPA Mix: {int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'FPA Mix: {int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'BKP Mix: {int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))}')
print(f'TAX Mix: {int(np.round(tax_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')

print(f'Fixed Mix: {int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))}')
print(f'Hourly Mix: {int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))}')


output_df['fsv_annualized'] = output_df.apply(lambda x: (x.increment/fsv_ot_start_percent)/current_month if x.frequency == 'one_time' else (x.increment*12)/current_month, axis=1)
output_df.fsv_annualized.to_clipboard()


Total FSV signed for the month: 3805312.5000000056
Total OT FSV signed for the month: 570796.8750000057
Total Rec FSV signed for the month: 3234515.625
--------------------------------------------------------------
Total FSV launched for the month: 2370535.5536514763
--------------------------------------------------------------
New OT FSV applicable for month: 199778.90625000198
New Rec FSV applicable for month: 134771.484375
--------------------------------------------------------------
Carryover OT IR for month: 213925.49021731736
Carryover Rec IR applicable for month: 242981.89422056975
--------------------------------------------------------------
New OT IR for month: 147511.36664778972
New Rec IR applicable for month: 108110.33972059788
--------------------------------------------------------------
OT Mix: 15
Rec Mix: 85
--------------------------------------------------------------
CFO Mix: 31, CFO FSV: 580673.0622815767
CPA Mix: 23
FPA Mix: 5
BKP Mix: 33
TAX Mix: 8
------------

In [345]:
output_df

,Optimized,Original,Month_Orig,increment,type,frequency,sl,rate
0,37001.493347,23723.886207,1000.000000,13277.607140,one_time_cfo_fixed,one_time,cfo,fixed
1,58186.123735,45405.469718,24310.000000,12780.654016,one_time_cfo_hourly,one_time,cfo,hourly
2,44065.670758,30622.017972,6300.000000,13443.652786,one_time_cpaaccounting_advisory_fixed,one_time,cpa,fixed
3,61162.831349,48120.250088,23494.932962,13042.581261,one_time_cpaaccounting_advisory_hourly,one_time,cpa,hourly
4,30881.213775,18860.241059,1000.000000,12020.972716,one_time_fpa_fixed,one_time,fpa,fixed
5,33088.285614,21847.119732,5552.500000,11241.165882,one_time_fpa_hourly,one_time,fpa,hourly
6,35888.739761,24062.752177,400.000000,11825.987584,one_time_full_charge_bookkeeping_fixed,one_time,bkp,fixed
7,38482.031011,25098.222991,4311.613211,13383.808020,one_time_full_charge_bookkeeping_hourly,one_time,bkp,hourly
8,43152.496976,30925.599968,8730.000000,12226.897007,one_time_tax_preparation_fixed,one_time,tax,fixed
9,1169.914161,1169.914161,1169.914161,0.000000,one_time_tax_preparation_hourly,one_time,tax,hourly


In [170]:
objective_fn(output_df.Original)

-1473.243234235619

In [171]:
objective_fn(output_df.Month_Orig)

-1136.80510000502

## August

In [56]:
## Constraints

x_0_date = '2021-08-01'
x_0 = x_0_transform.filter(like = x_0_date, axis=0).values[0].tolist()
x_0_orig = x_0
churn_rate = 0.9
# adding in projects already launched
x_0 = [(x_0[idx] + july_additions[idx]) if idx <10 else (x_0[idx] + (churn_rate**2)*june_additions[idx] + (churn_rate)*july_additions[idx]) for idx,value in enumerate(x_0) ]

# adding projects not yet launched
x_0 = [(x_0[idx] + june_additions[idx]*two_months_back/current_month + july_additions[idx]*previous_month/current_month) for idx,value in enumerate(x_0) ]


fsv_target = 4462500 #August
fsv_previous_month = 4615625 #July
fsv_2_months_back = 4118750 #June
fsv_3_months_back = 0

one_time_mix_lower = 0.09
one_time_mix_upper = 0.15

recurring_mix_lower = 0.5
recurring_mix_upper = 0.8

cfo_mix_lower = 0.25
cfo_mix_upper = 0.35

cpa_mix_lower = 0.21
cpa_mix_upper = 0.32

fpa_mix_lower = 0.03
fpa_mix_upper = 0.1

bkp_mix_lower = 0.5
bkp_mix_upper = 0.26

tax_mix_lower = 0.05
tax_mix_upper = 0.08

fixed_mix_lower = 0.3
fixed_mix_upper = 0.7

hourly_mix_lower = 0.15
hourly_mix_upper = 0.4

#Realization ratios
current_month = 0.45
previous_month = 0.25
two_months_back = 0.1
three_months_back = 0.05


fsv_ot_start_percent = 0.7

target_achieve_factor = 0.9
target_miss_factor = 0.8


fsv_to_start = current_month*fsv_target #+ previous_month*fsv_previous_month + two_months_back*fsv_2_months_back + three_months_back*fsv_3_months_back
fsv_to_start = fsv_to_start*target_achieve_factor

upper_limit_one_time_pr_lower = fsv_to_start*one_time_mix_lower


def total_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return fsv_to_start - (ot_annual_fsv + rec_annual_fsv)

def total_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (ot_annual_fsv + rec_annual_fsv) - target_miss_factor*fsv_to_start

def one_time_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (fsv_to_start)*one_time_mix_upper - ot_annual_fsv

def one_time_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return ot_annual_fsv - (fsv_to_start)*one_time_mix_lower 

def hourly_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return fsv_to_start*hourly_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def hourly_fsv_constraint_lower(x):
    
    current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
    additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper
    
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*hourly_mix_lower

def cfo_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*cfo_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def cfo_fsv_constraint_lower(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*cfo_mix_lower

def fpa_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.fpa.value][0:2]) - sum(np.array(x_0)[Mix.fpa.value][0:2])
    rec_addition = sum(np.array(x)[Mix.fpa.value][2:]) - sum(np.array(x_0)[Mix.fpa.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*fpa_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def tax_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.tax.value][0:2]) - sum(np.array(x_0)[Mix.tax.value][0:2])
    rec_addition = sum(np.array(x)[Mix.tax.value][2:]) - sum(np.array(x_0)[Mix.tax.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*tax_mix_upper - (ot_annual_fsv + rec_annual_fsv)  
    
total_fsv_cons = {'type':'ineq', 'fun':total_fsv_constraint}
total_fsv_cons_lower = {'type':'ineq', 'fun':total_fsv_constraint_lower}
ot_fsv_cons = {'type':'ineq', 'fun':one_time_fsv_constraint}
ot_fsv_cons_lower = {'type':'ineq', 'fun':one_time_fsv_constraint_lower}

hourly_fsv_cons = {'type':'ineq', 'fun':hourly_fsv_constraint}
hourly_fsv_cons_lower = {'type':'ineq', 'fun':hourly_fsv_constraint_lower}
cfo_fsv_cons = {'type':'ineq', 'fun':cfo_fsv_constraint}
cfo_fsv_cons_lower = {'type':'ineq', 'fun':cfo_fsv_constraint_lower}
fpa_fsv_cons = {'type':'ineq', 'fun':fpa_fsv_constraint}
tax_fsv_cons = {'type':'ineq', 'fun':tax_fsv_constraint}

constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, ot_fsv_cons_lower, hourly_fsv_cons, hourly_fsv_cons_lower, cfo_fsv_cons, cfo_fsv_cons_lower, fpa_fsv_cons, tax_fsv_cons ]

bounds = ()
for idx, val in enumerate(x_0):
    bound_val = (x_0[idx] *1.05,500000)
    bounds = bounds + (bound_val,)
    
trend = 41
print_mode = 0
opt_method = 'COBYLA'
# opt_method = 'L-BFGS-B'
opt_method = 'SLSQP'
sol = minimize(objective_fn, x_0, method=opt_method, bounds = bounds, constraints=constraints, options={'maxiter': 500000, 'disp': True ,'eps' : 0.5})

output_df = pd.DataFrame([sol.x, x_0, x_0_orig]).T
output_df.columns = ['Optimized', 'Original', 'Month_Orig']
output_df['Optimized'] = output_df.apply(lambda x: x.Optimized if x.Optimized > x.Original else x.Original, axis=1)
output_df['increment'] = output_df['Optimized'] - output_df['Original']
output_df['type'] = output_columns
output_df['frequency'] = output_frequency
output_df['sl'] = output_sl
output_df['rate'] = output_rate_type
august_additions = list(output_df['increment'])
print_mode = 0
print(f'New FSV for the month: {1.11*(sum(list(output_df.Optimized)) - sum(list(output_df.Original)))}')
print(f'New IR for the month: {-0.85*1000*(objective_fn(list(output_df.Optimized)) - objective_fn(list(output_df.Original)))}')
print(f'Total FSV for the month: {1*sum(list(output_df.Optimized))}')
print(sol)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2187.125743069336
            Iterations: 78
            Function evaluations: 1618
            Gradient evaluations: 77
New FSV for the month: 352742.21718750003
New IR for the month: 311997.8154300627
Total FSV for the month: 2351337.2454905743
     fun: -2187.125743069336
     jac: array([-0.0007232 , -0.00075901, -0.00080958, -0.00082965, -0.0006554 ,
       -0.00063611, -0.00075163, -0.00071127, -0.00080723,  0.01217919,
       -0.00169706, -0.00135758, -0.0018573 , -0.00143042, -0.00112472,
       -0.00035639, -0.00226014, -0.00152682, -0.00331223, -0.00288775])
 message: 'Optimization terminated successfully'
    nfev: 1618
     nit: 78
    njev: 77
  status: 0
 success: True
       x: array([ 61678.15708991,  65099.17377235,  67218.97273868,  72253.17974877,
        54146.22107836,  50890.09231899,  60110.44076206,  58786.03576403,
        62519.6556729 ,    976.17201326, 251520.68514134,

In [57]:
ot_fsv_to_start = (sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/fsv_ot_start_percent
rec_fsv_to_start = (sum(list(output_df.Optimized)[10:]) - sum(list(output_df.Original)[10:]))*12
total_fsv_to_start = ot_fsv_to_start + rec_fsv_to_start

(ot_opt, rec_opt) = objective_fn_forecast(output_df.Optimized)
(ot, rec) = objective_fn_forecast(output_df.Month_Orig)
(ot_new, rec_new) = objective_fn_forecast(output_df.Original)


print(f'Total FSV signed for the month: {total_fsv_to_start/current_month}')
print(f'Total OT FSV signed for the month: {ot_fsv_to_start/current_month}')
print(f'Total Rec FSV signed for the month: {rec_fsv_to_start/current_month}')
print('--------------------------------------------------------------')
print(f'Total FSV launched for the month: {1.07*(sum(list(output_df.Optimized)))}')
# print(f'Total OT FSV launched for the month: {ot_fsv_to_start}')
# print(f'Total Rec FSV launched for the month: {rec_fsv_to_start}')
print('--------------------------------------------------------------')
print(f'New OT FSV applicable for month: {ot_fsv_to_start*fsv_ot_start_percent}')
print(f'New Rec FSV applicable for month: {rec_fsv_to_start/12}')
print('--------------------------------------------------------------')
print(f'Carryover OT IR for month: {ot_new - ot}')
print(f'Carryover Rec IR applicable for month: {(rec_new - rec)}')
print('--------------------------------------------------------------')
print(f'New OT IR for month: {ot_opt - ot_new}')
print(f'New Rec IR applicable for month: {0.45*(rec_opt - rec_new)}')
print('--------------------------------------------------------------')


cfo_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
cpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
fpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
bkp_fsv_to_start = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
tax_fsv_to_start = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12

fixed_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fixed.value])[0:5]) - sum(list(output_df.Original[Mix.fixed.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fixed.value])[5:]) - sum(list(output_df.Original[Mix.fixed.value])[5:]))*12
hourly_fsv_to_start = ((sum(list(output_df.Optimized[Mix.hourly.value])[0:5]) - sum(list(output_df.Original[Mix.hourly.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.hourly.value])[5:]) - sum(list(output_df.Original[Mix.hourly.value])[5:]))*12


print(f'OT Mix: {int(np.round(ot_fsv_to_start*100/total_fsv_to_start))}')
print(f'Rec Mix: {int(np.round(rec_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')
print(f'CFO Mix: {int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))}, CFO FSV: {cfo_fsv_to_start}')
print(f'CPA Mix: {int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'FPA Mix: {int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'BKP Mix: {int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))}')
print(f'TAX Mix: {int(np.round(tax_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')

print(f'Fixed Mix: {int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))}')
print(f'Hourly Mix: {int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))}')


output_df['fsv_annualized'] = output_df.apply(lambda x: (x.increment/fsv_ot_start_percent)/current_month if x.frequency == 'one_time' else (x.increment*12)/current_month, axis=1)
output_df.fsv_annualized.to_clipboard()


Total FSV signed for the month: 4016250.0
Total OT FSV signed for the month: 602437.5
Total Rec FSV signed for the month: 3413812.5
--------------------------------------------------------------
Total FSV launched for the month: 2515930.8526749145
--------------------------------------------------------------
New OT FSV applicable for month: 189767.8125
New Rec FSV applicable for month: 128017.96875
--------------------------------------------------------------
Carryover OT IR for month: 262376.4933287197
Carryover Rec IR applicable for month: 456192.5173116947
--------------------------------------------------------------
New OT IR for month: 141568.3365927835
New Rec IR applicable for month: 101469.56258445715
--------------------------------------------------------------
OT Mix: 15
Rec Mix: 85
--------------------------------------------------------------
CFO Mix: 26, CFO FSV: 474398.10880174447
CPA Mix: 23
FPA Mix: 5
BKP Mix: 37
TAX Mix: 8
------------------------------------------

In [196]:
5125155.62009627/12
#fsv_to_start
sum(list(output_df.Optimized)[10:])
rec_fsv_to_start

2306320.0290433215

## September

In [58]:
## Constraints

x_0_date = '2021-09-01'
x_0 = x_0_transform.filter(like = x_0_date, axis=0).values[0].tolist()
x_0_orig = x_0
churn_rate = 0.9
# adding in projects already launched
x_0 = [(x_0[idx] + august_additions[idx]) if idx <10 else (x_0[idx] + (churn_rate**3)*june_additions[idx] + (churn_rate**2)*july_additions[idx] + (churn_rate)*august_additions[idx]) for idx,value in enumerate(x_0) ]

# adding projects not yet launched
x_0 = [(x_0[idx] + june_additions[idx]*three_months_back/current_month + july_additions[idx]*two_months_back/current_month + august_additions[idx]*previous_month/current_month) for idx,value in enumerate(x_0) ]


fsv_target = 4659375 #September
fsv_previous_month = 5209375 #August
fsv_2_months_back = 4615625 #July
fsv_3_months_back = 4118750 #June

one_time_mix_lower = 0.09
one_time_mix_upper = 0.15

recurring_mix_lower = 0.5
recurring_mix_upper = 0.8

cfo_mix_lower = 0.25
cfo_mix_upper = 0.35

cpa_mix_lower = 0.21
cpa_mix_upper = 0.32

fpa_mix_lower = 0.03
fpa_mix_upper = 0.1

bkp_mix_lower = 0.5
bkp_mix_upper = 0.26

tax_mix_lower = 0.05
tax_mix_upper = 0.08

fixed_mix_lower = 0.3
fixed_mix_upper = 0.7

hourly_mix_lower = 0.15
hourly_mix_upper = 0.4

#Realization ratios
current_month = 0.5
previous_month = 0.25
two_months_back = 0.1
three_months_back = 0.05


fsv_ot_start_percent = 0.7

target_achieve_factor = 0.9
target_miss_factor = 0.8


fsv_to_start = current_month*fsv_target #+ previous_month*fsv_previous_month + two_months_back*fsv_2_months_back + three_months_back*fsv_3_months_back
fsv_to_start = fsv_to_start*target_achieve_factor

upper_limit_one_time_pr_lower = fsv_to_start*one_time_mix_lower


def total_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return fsv_to_start - (ot_annual_fsv + rec_annual_fsv)

def total_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (ot_annual_fsv + rec_annual_fsv) - target_miss_factor*fsv_to_start

def one_time_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (fsv_to_start)*one_time_mix_upper - ot_annual_fsv

def one_time_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return ot_annual_fsv - (fsv_to_start)*one_time_mix_lower 

def hourly_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return fsv_to_start*hourly_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def hourly_fsv_constraint_lower(x):
    
    current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
    additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper
    
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*hourly_mix_lower

def cfo_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*cfo_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def cfo_fsv_constraint_lower(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*cfo_mix_lower

def fpa_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.fpa.value][0:2]) - sum(np.array(x_0)[Mix.fpa.value][0:2])
    rec_addition = sum(np.array(x)[Mix.fpa.value][2:]) - sum(np.array(x_0)[Mix.fpa.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*fpa_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def tax_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.tax.value][0:2]) - sum(np.array(x_0)[Mix.tax.value][0:2])
    rec_addition = sum(np.array(x)[Mix.tax.value][2:]) - sum(np.array(x_0)[Mix.tax.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*tax_mix_upper - (ot_annual_fsv + rec_annual_fsv)  
    
total_fsv_cons = {'type':'ineq', 'fun':total_fsv_constraint}
total_fsv_cons_lower = {'type':'ineq', 'fun':total_fsv_constraint_lower}
ot_fsv_cons = {'type':'ineq', 'fun':one_time_fsv_constraint}
ot_fsv_cons_lower = {'type':'ineq', 'fun':one_time_fsv_constraint_lower}

hourly_fsv_cons = {'type':'ineq', 'fun':hourly_fsv_constraint}
hourly_fsv_cons_lower = {'type':'ineq', 'fun':hourly_fsv_constraint_lower}
cfo_fsv_cons = {'type':'ineq', 'fun':cfo_fsv_constraint}
cfo_fsv_cons_lower = {'type':'ineq', 'fun':cfo_fsv_constraint_lower}
fpa_fsv_cons = {'type':'ineq', 'fun':fpa_fsv_constraint}
tax_fsv_cons = {'type':'ineq', 'fun':tax_fsv_constraint}

constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, ot_fsv_cons_lower, hourly_fsv_cons, hourly_fsv_cons_lower, cfo_fsv_cons, cfo_fsv_cons_lower, fpa_fsv_cons, tax_fsv_cons ]

bounds = ()
for idx, val in enumerate(x_0):
    bound_val = (x_0[idx] *1.05,500000)
    bounds = bounds + (bound_val,)
    
trend = 41
print_mode = 0
opt_method = 'COBYLA'
# opt_method = 'L-BFGS-B'
opt_method = 'SLSQP'
sol = minimize(objective_fn, x_0, method=opt_method, bounds = bounds, constraints=constraints, options={'maxiter': 500000, 'disp': True ,'eps' : 0.5})

output_df = pd.DataFrame([sol.x, x_0, x_0_orig]).T
output_df.columns = ['Optimized', 'Original', 'Month_Orig']
output_df['Optimized'] = output_df.apply(lambda x: x.Optimized if x.Optimized > x.Original else x.Original, axis=1)
output_df['increment'] = output_df['Optimized'] - output_df['Original']
output_df['type'] = output_columns
output_df['frequency'] = output_frequency
output_df['sl'] = output_sl
output_df['rate'] = output_rate_type
september_additions = list(1*output_df['increment'])
print_mode = 0
print(f'New FSV for the month: {1.11*(sum(list(output_df.Optimized)) - sum(list(output_df.Original)))}')
print(f'New IR for the month: {-0.85*1000*(objective_fn(list(output_df.Optimized)) - objective_fn(list(output_df.Original)))}')
print(f'Total FSV for the month: {1*sum(list(output_df.Optimized))}')
print(sol)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2499.963367009078
            Iterations: 79
            Function evaluations: 1649
            Gradient evaluations: 78
New FSV for the month: 409227.0820312505
New IR for the month: 396609.9903149432
Total FSV for the month: 2534470.150616097
     fun: -2499.963367009078
     jac: array([-0.00072376, -0.00075689, -0.00080503, -0.00082397, -0.00066434,
       -0.00064753, -0.00074935, -0.00071226, -0.00080797,  0.01239643,
       -0.00197906, -0.00161043, -0.00215637, -0.00169472, -0.00134367,
       -0.00049097, -0.0025909 , -0.00180181, -0.00360414, -0.00312376])
 message: 'Optimization terminated successfully'
    nfev: 1649
     nit: 79
    njev: 78
  status: 0
 success: True
       x: array([ 68084.29135762,  68806.72422612,  72400.60695211,  75361.18940322,
        64194.45547567,  56632.54609939,  66043.13580131,  63360.41506852,
        61948.00290376,    959.94153152, 276217.34133235, 2

In [59]:
ot_fsv_to_start = (sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/fsv_ot_start_percent
rec_fsv_to_start = (sum(list(output_df.Optimized)[10:]) - sum(list(output_df.Original)[10:]))*12
total_fsv_to_start = ot_fsv_to_start + rec_fsv_to_start

(ot_opt, rec_opt) = objective_fn_forecast(output_df.Optimized)
(ot, rec) = objective_fn_forecast(output_df.Month_Orig)
(ot_new, rec_new) = objective_fn_forecast(output_df.Original)


print(f'Total FSV signed for the month: {total_fsv_to_start/current_month}')
print(f'Total OT FSV signed for the month: {ot_fsv_to_start/current_month}')
print(f'Total Rec FSV signed for the month: {rec_fsv_to_start/current_month}')
print('--------------------------------------------------------------')
print(f'Total FSV launched for the month: {1.07*(sum(list(output_df.Optimized)))}')
# print(f'Total OT FSV launched for the month: {ot_fsv_to_start}')
# print(f'Total Rec FSV launched for the month: {rec_fsv_to_start}')
print('--------------------------------------------------------------')
print(f'New OT FSV applicable for month: {ot_fsv_to_start*fsv_ot_start_percent}')
print(f'New Rec FSV applicable for month: {rec_fsv_to_start/12}')
print('--------------------------------------------------------------')
print(f'Carryover OT IR for month: {ot_new - ot}')
print(f'Carryover Rec IR applicable for month: {(rec_new - rec)}')
print('--------------------------------------------------------------')
print(f'New OT IR for month: {ot_opt - ot_new}')
print(f'New Rec IR applicable for month: {0.4*(rec_opt - rec_new)}')
print('--------------------------------------------------------------')


cfo_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
cpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
fpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
bkp_fsv_to_start = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
tax_fsv_to_start = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12

fixed_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fixed.value])[0:5]) - sum(list(output_df.Original[Mix.fixed.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fixed.value])[5:]) - sum(list(output_df.Original[Mix.fixed.value])[5:]))*12
hourly_fsv_to_start = ((sum(list(output_df.Optimized[Mix.hourly.value])[0:5]) - sum(list(output_df.Original[Mix.hourly.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.hourly.value])[5:]) - sum(list(output_df.Original[Mix.hourly.value])[5:]))*12


print(f'OT Mix: {int(np.round(ot_fsv_to_start*100/total_fsv_to_start))}')
print(f'Rec Mix: {int(np.round(rec_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')
print(f'CFO Mix: {int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))}, CFO FSV: {cfo_fsv_to_start}')
print(f'CPA Mix: {int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'FPA Mix: {int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'BKP Mix: {int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))}')
print(f'TAX Mix: {int(np.round(tax_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')

print(f'Fixed Mix: {int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))}')
print(f'Hourly Mix: {int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))}')


output_df['fsv_annualized'] = output_df.apply(lambda x: (x.increment/fsv_ot_start_percent)/current_month if x.frequency == 'one_time' else (x.increment*12)/current_month, axis=1)
output_df.fsv_annualized.to_clipboard()


Total FSV signed for the month: 4193437.500000001
Total OT FSV signed for the month: 629015.6250000007
Total Rec FSV signed for the month: 3564421.875
--------------------------------------------------------------
Total FSV launched for the month: 2711883.061159224
--------------------------------------------------------------
New OT FSV applicable for month: 220155.46875000023
New Rec FSV applicable for month: 148517.578125
--------------------------------------------------------------
Carryover OT IR for month: 323582.36018766847
Carryover Rec IR applicable for month: 657871.5054061002
--------------------------------------------------------------
New OT IR for month: 164644.73095351364
New Rec IR applicable for month: 120782.10306092064
--------------------------------------------------------------
OT Mix: 15
Rec Mix: 85
--------------------------------------------------------------
CFO Mix: 27, CFO FSV: 557440.2925994138
CPA Mix: 23
FPA Mix: 5
BKP Mix: 37
TAX Mix: 8
---------------

In [425]:
np.random.randint(40,50)/100

0.48

## October

In [60]:
## Constraints

x_0_date = '2021-10-01'
x_0 = x_0_transform.filter(like = x_0_date, axis=0).values[0].tolist()
x_0_orig = x_0
churn_rate = 0.9

# adding in projects already launched
x_0 = [(x_0[idx] + september_additions[idx]) if idx <10 else (x_0[idx] + (churn_rate**4)*june_additions[idx] + (churn_rate**3)*july_additions[idx] 
                                                              + (churn_rate**2)*august_additions[idx] + (churn_rate**1)*september_additions[idx]) for idx,value in enumerate(x_0) ]

# adding projects not yet launched
x_0 = [(x_0[idx] + july_additions[idx]*three_months_back/current_month + august_additions[idx]*two_months_back/current_month 
        + september_additions[idx]*previous_month/current_month) for idx,value in enumerate(x_0) ]

fsv_target = 4893750 #October
fsv_previous_month = 5743750 #September
fsv_2_months_back = 5209375 #August
fsv_3_months_back = 4615625 #July

one_time_mix_lower = 0.09
one_time_mix_upper = 0.15

recurring_mix_lower = 0.5
recurring_mix_upper = 0.8

cfo_mix_lower = 0.25
cfo_mix_upper = 0.35

cpa_mix_lower = 0.21
cpa_mix_upper = 0.32

fpa_mix_lower = 0.03
fpa_mix_upper = 0.1

bkp_mix_lower = 0.5
bkp_mix_upper = 0.26

tax_mix_lower = 0.05
tax_mix_upper = 0.08

fixed_mix_lower = 0.3
fixed_mix_upper = 0.7

hourly_mix_lower = 0.15
hourly_mix_upper = 0.4

#Realization ratios
current_month = 0.5
previous_month = 0.25
two_months_back = 0.1
three_months_back = 0.05


fsv_ot_start_percent = 0.7

target_achieve_factor = 0.9
target_miss_factor = 0.8


fsv_to_start = current_month*fsv_target #+ previous_month*fsv_previous_month + two_months_back*fsv_2_months_back + three_months_back*fsv_3_months_back
fsv_to_start = fsv_to_start*target_achieve_factor

upper_limit_one_time_pr_lower = fsv_to_start*one_time_mix_lower


def total_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return fsv_to_start - (ot_annual_fsv + rec_annual_fsv)

def total_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (ot_annual_fsv + rec_annual_fsv) - target_miss_factor*fsv_to_start

def one_time_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (fsv_to_start)*one_time_mix_upper - ot_annual_fsv

def one_time_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return ot_annual_fsv - (fsv_to_start)*one_time_mix_lower 

def hourly_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return fsv_to_start*hourly_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def hourly_fsv_constraint_lower(x):
    
    current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
    additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper
    
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*hourly_mix_lower

def cfo_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*cfo_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def cfo_fsv_constraint_lower(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*cfo_mix_lower

def fpa_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.fpa.value][0:2]) - sum(np.array(x_0)[Mix.fpa.value][0:2])
    rec_addition = sum(np.array(x)[Mix.fpa.value][2:]) - sum(np.array(x_0)[Mix.fpa.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*fpa_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def tax_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.tax.value][0:2]) - sum(np.array(x_0)[Mix.tax.value][0:2])
    rec_addition = sum(np.array(x)[Mix.tax.value][2:]) - sum(np.array(x_0)[Mix.tax.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*tax_mix_upper - (ot_annual_fsv + rec_annual_fsv)  
    
total_fsv_cons = {'type':'ineq', 'fun':total_fsv_constraint}
total_fsv_cons_lower = {'type':'ineq', 'fun':total_fsv_constraint_lower}
ot_fsv_cons = {'type':'ineq', 'fun':one_time_fsv_constraint}
ot_fsv_cons_lower = {'type':'ineq', 'fun':one_time_fsv_constraint_lower}

hourly_fsv_cons = {'type':'ineq', 'fun':hourly_fsv_constraint}
hourly_fsv_cons_lower = {'type':'ineq', 'fun':hourly_fsv_constraint_lower}
cfo_fsv_cons = {'type':'ineq', 'fun':cfo_fsv_constraint}
cfo_fsv_cons_lower = {'type':'ineq', 'fun':cfo_fsv_constraint_lower}
fpa_fsv_cons = {'type':'ineq', 'fun':fpa_fsv_constraint}
tax_fsv_cons = {'type':'ineq', 'fun':tax_fsv_constraint}

constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, ot_fsv_cons_lower, hourly_fsv_cons, hourly_fsv_cons_lower, cfo_fsv_cons, cfo_fsv_cons_lower, fpa_fsv_cons, tax_fsv_cons ]

bounds = ()
for idx, val in enumerate(x_0):
    bound_val = (x_0[idx] *1.05,700000)
    bounds = bounds + (bound_val,)
    
trend = 41
print_mode = 0
opt_method = 'COBYLA'
# opt_method = 'L-BFGS-B'
opt_method = 'SLSQP'
sol = minimize(objective_fn, x_0, method=opt_method, bounds = bounds, constraints=constraints, options={'maxiter': 500000, 'disp': True ,'eps' : 0.5})

output_df = pd.DataFrame([sol.x, x_0, x_0_orig]).T
output_df.columns = ['Optimized', 'Original', 'Month_Orig']
output_df['Optimized'] = output_df.apply(lambda x: x.Optimized if x.Optimized > x.Original else x.Original, axis=1)
output_df['increment'] = output_df['Optimized'] - output_df['Original']
output_df['type'] = output_columns
output_df['frequency'] = output_frequency
output_df['sl'] = output_sl
output_df['rate'] = output_rate_type
october_additions = list(1*output_df['increment'])
print_mode = 0
print(f'New FSV for the month: {1.11*(sum(list(output_df.Optimized)) - sum(list(output_df.Original)))}')
print(f'New IR for the month: {-1*1000*(objective_fn(list(output_df.Optimized)) - objective_fn(list(output_df.Original)))}')
print(f'Total FSV for the month: {1.11*1.05*sum(list(output_df.Optimized))}')
print(sol)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2778.689120433377
            Iterations: 99
            Function evaluations: 2068
            Gradient evaluations: 98
New FSV for the month: 429811.94531248766
New IR for the month: 528590.1052425047
Total FSV for the month: 3113888.7667886526
     fun: -2778.689120433377
     jac: array([-0.00072398, -0.00075509, -0.00080511, -0.00082401, -0.00066735,
       -0.00065135, -0.00074788, -0.00071296, -0.00080272,  0.01416886,
       -0.00223683, -0.00183974, -0.00242894, -0.00192796, -0.0015422 ,
       -0.00060961, -0.00289128, -0.00205764, -0.00389577, -0.00336033])
 message: 'Optimization terminated successfully'
    nfev: 2068
     nit: 99
    njev: 98
  status: 0
 success: True
       x: array([ 73038.16441302,  72788.70163977,  74182.23595775,  73830.9407627 ,
        65418.91645717,  61173.16170337,  70526.3116778 ,  66970.79167294,
        65918.9388778 ,    844.36440693, 296916.34017866,

In [61]:
ot_fsv_to_start = (sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/fsv_ot_start_percent
rec_fsv_to_start = (sum(list(output_df.Optimized)[10:]) - sum(list(output_df.Original)[10:]))*12
total_fsv_to_start = ot_fsv_to_start + rec_fsv_to_start

(ot_opt, rec_opt) = objective_fn_forecast(output_df.Optimized)
(ot, rec) = objective_fn_forecast(output_df.Month_Orig)
(ot_new, rec_new) = objective_fn_forecast(output_df.Original)


print(f'Total FSV signed for the month: {total_fsv_to_start/current_month}')
print(f'Total OT FSV signed for the month: {ot_fsv_to_start/current_month}')
print(f'Total Rec FSV signed for the month: {rec_fsv_to_start/current_month}')
print('--------------------------------------------------------------')
print(f'Total FSV launched for the month: {1.07*(sum(list(output_df.Optimized)))}')
# print(f'Total OT FSV launched for the month: {ot_fsv_to_start}')
# print(f'Total Rec FSV launched for the month: {rec_fsv_to_start}')
print('--------------------------------------------------------------')
print(f'New OT FSV applicable for month: {ot_fsv_to_start*fsv_ot_start_percent}')
print(f'New Rec FSV applicable for month: {rec_fsv_to_start/12}')
print('--------------------------------------------------------------')
print(f'Carryover OT IR for month: {ot_new - ot}')
print(f'Carryover Rec IR applicable for month: {(rec_new - rec)}')
print('--------------------------------------------------------------')
print(f'New OT IR for month: {ot_opt - ot_new}')
print(f'New Rec IR applicable for month: {0.35*(rec_opt - rec_new)}')
print('--------------------------------------------------------------')


cfo_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
cpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
fpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
bkp_fsv_to_start = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
tax_fsv_to_start = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12

fixed_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fixed.value])[0:5]) - sum(list(output_df.Original[Mix.fixed.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fixed.value])[5:]) - sum(list(output_df.Original[Mix.fixed.value])[5:]))*12
hourly_fsv_to_start = ((sum(list(output_df.Optimized[Mix.hourly.value])[0:5]) - sum(list(output_df.Original[Mix.hourly.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.hourly.value])[5:]) - sum(list(output_df.Original[Mix.hourly.value])[5:]))*12


print(f'OT Mix: {int(np.round(ot_fsv_to_start*100/total_fsv_to_start))}')
print(f'Rec Mix: {int(np.round(rec_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')
print(f'CFO Mix: {int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))}, CFO FSV: {cfo_fsv_to_start}')
print(f'CPA Mix: {int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'FPA Mix: {int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'BKP Mix: {int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))}')
print(f'TAX Mix: {int(np.round(tax_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')

print(f'Fixed Mix: {int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))}')
print(f'Hourly Mix: {int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))}')


output_df['fsv_annualized'] = output_df.apply(lambda x: (x.increment/fsv_ot_start_percent)/current_month if x.frequency == 'one_time' else (x.increment*12)/current_month, axis=1)
output_df.fsv_annualized.to_clipboard()


Total FSV signed for the month: 4404374.999999111
Total OT FSV signed for the month: 660656.250000083
Total Rec FSV signed for the month: 3743718.7499990277
--------------------------------------------------------------
Total FSV launched for the month: 2858739.57997757
--------------------------------------------------------------
New OT FSV applicable for month: 231229.68750002905
New Rec FSV applicable for month: 155988.2812499595
--------------------------------------------------------------
Carryover OT IR for month: 326091.95290838013
Carryover Rec IR applicable for month: 859215.3762750372
--------------------------------------------------------------
New OT IR for month: 172928.42016855814
New Rec IR applicable for month: 124481.58977588131
--------------------------------------------------------------
OT Mix: 15
Rec Mix: 85
--------------------------------------------------------------
CFO Mix: 26, CFO FSV: 562598.0083928521
CPA Mix: 22
FPA Mix: 5
BKP Mix: 39
TAX Mix: 8
------

## November

In [64]:
## Constraints

x_0_date = '2021-11-01'
x_0 = x_0_transform.filter(like = x_0_date, axis=0).values[0].tolist()

x_0_orig = x_0
churn_rate = 0.9

# adding in projects already launched
x_0 = [(x_0[idx] + october_additions[idx]) if idx <10 else (x_0[idx] + (churn_rate**5)*june_additions[idx] + (churn_rate**4)*july_additions[idx] 
                                                              + (churn_rate**3)*august_additions[idx] + (churn_rate**2)*september_additions[idx] 
                                                            + (churn_rate**1)*october_additions[idx]) for idx,value in enumerate(x_0) ]

# adding projects not yet launched
x_0 = [(x_0[idx] + august_additions[idx]*three_months_back/current_month + september_additions[idx]*two_months_back/current_month 
        + october_additions[idx]*previous_month/current_month) for idx,value in enumerate(x_0) ]



fsv_target = 5253125 #November
fsv_previous_month = 6115625 #October
fsv_2_months_back = 5743750 #September
fsv_3_months_back = 5209375 #August

one_time_mix_lower = 0.09
one_time_mix_upper = 0.15

recurring_mix_lower = 0.5
recurring_mix_upper = 0.8

cfo_mix_lower = 0.25
cfo_mix_upper = 0.35

cpa_mix_lower = 0.21
cpa_mix_upper = 0.32

fpa_mix_lower = 0.03
fpa_mix_upper = 0.1

bkp_mix_lower = 0.5
bkp_mix_upper = 0.26

tax_mix_lower = 0.05
tax_mix_upper = 0.08

fixed_mix_lower = 0.3
fixed_mix_upper = 0.7

hourly_mix_lower = 0.15
hourly_mix_upper = 0.4

#Realization ratios
current_month = 0.5
previous_month = 0.25
two_months_back = 0.1
three_months_back = 0.05


fsv_ot_start_percent = 0.7

target_achieve_factor = 0.9
target_miss_factor = 0.8


fsv_to_start = current_month*fsv_target #+ previous_month*fsv_previous_month + two_months_back*fsv_2_months_back + three_months_back*fsv_3_months_back
fsv_to_start = fsv_to_start*target_achieve_factor

upper_limit_one_time_pr_lower = fsv_to_start*one_time_mix_lower


def total_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return fsv_to_start - (ot_annual_fsv + rec_annual_fsv)

def total_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (ot_annual_fsv + rec_annual_fsv) - target_miss_factor*fsv_to_start

def one_time_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (fsv_to_start)*one_time_mix_upper - ot_annual_fsv

def one_time_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return ot_annual_fsv - (fsv_to_start)*one_time_mix_lower 

def hourly_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return fsv_to_start*hourly_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def hourly_fsv_constraint_lower(x):
    
    current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
    additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper
    
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*hourly_mix_lower

def cfo_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*cfo_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def cfo_fsv_constraint_lower(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*cfo_mix_lower

def fpa_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.fpa.value][0:2]) - sum(np.array(x_0)[Mix.fpa.value][0:2])
    rec_addition = sum(np.array(x)[Mix.fpa.value][2:]) - sum(np.array(x_0)[Mix.fpa.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*fpa_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def tax_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.tax.value][0:2]) - sum(np.array(x_0)[Mix.tax.value][0:2])
    rec_addition = sum(np.array(x)[Mix.tax.value][2:]) - sum(np.array(x_0)[Mix.tax.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*tax_mix_upper - (ot_annual_fsv + rec_annual_fsv)  
    
total_fsv_cons = {'type':'ineq', 'fun':total_fsv_constraint}
total_fsv_cons_lower = {'type':'ineq', 'fun':total_fsv_constraint_lower}
ot_fsv_cons = {'type':'ineq', 'fun':one_time_fsv_constraint}
ot_fsv_cons_lower = {'type':'ineq', 'fun':one_time_fsv_constraint_lower}

hourly_fsv_cons = {'type':'ineq', 'fun':hourly_fsv_constraint}
hourly_fsv_cons_lower = {'type':'ineq', 'fun':hourly_fsv_constraint_lower}
cfo_fsv_cons = {'type':'ineq', 'fun':cfo_fsv_constraint}
cfo_fsv_cons_lower = {'type':'ineq', 'fun':cfo_fsv_constraint_lower}
fpa_fsv_cons = {'type':'ineq', 'fun':fpa_fsv_constraint}
tax_fsv_cons = {'type':'ineq', 'fun':tax_fsv_constraint}

constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, ot_fsv_cons_lower, hourly_fsv_cons, hourly_fsv_cons_lower, cfo_fsv_cons, cfo_fsv_cons_lower, fpa_fsv_cons, tax_fsv_cons ]

bounds = ()
for idx, val in enumerate(x_0):
    bound_val = (x_0[idx] *1.05,700000)
    bounds = bounds + (bound_val,)
    
trend = 41
print_mode = 0
opt_method = 'COBYLA'
# opt_method = 'L-BFGS-B'
opt_method = 'SLSQP'
sol = minimize(objective_fn, x_0, method=opt_method, bounds = bounds, constraints=constraints, options={'maxiter': 500000, 'disp': True ,'eps' : 0.5})

output_df = pd.DataFrame([sol.x, x_0, x_0_orig]).T
output_df.columns = ['Optimized', 'Original', 'Month_Orig']
output_df['Optimized'] = output_df.apply(lambda x: x.Optimized if x.Optimized > x.Original else x.Original, axis=1)
output_df['increment'] = output_df['Optimized'] - output_df['Original']
output_df['type'] = output_columns
output_df['frequency'] = output_frequency
output_df['sl'] = output_sl
output_df['rate'] = output_rate_type
november_additions = list(1*output_df['increment'])
print_mode = 0
print(f'New FSV for the month: {1.11*(sum(list(output_df.Optimized)) - sum(list(output_df.Original)))}')
print(f'New IR for the month: {-1*1000*(objective_fn(list(output_df.Optimized)) - objective_fn(list(output_df.Original)))}')
print(f'Total FSV for the month: {1.11*1.05*sum(list(output_df.Optimized))}')
print(sol)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3170.9869658285343
            Iterations: 96
            Function evaluations: 1996
            Gradient evaluations: 95
New FSV for the month: 461375.40234374796
New IR for the month: 623081.3040298622
Total FSV for the month: 3332259.7510917247
     fun: -3170.9869658285343
     jac: array([-0.00072421, -0.00075274, -0.00079854, -0.00081587, -0.00067264,
       -0.00065809, -0.00074592, -0.00071402, -0.0007976 ,  0.01427969,
       -0.00258748, -0.00215114, -0.00279883, -0.0022443 , -0.00181431,
       -0.00077573, -0.00329709, -0.00239808, -0.00430643, -0.00369742])
 message: 'Optimization terminated successfully'
    nfev: 1996
     nit: 96
    njev: 95
  status: 0
 success: True
       x: array([ 80122.96257282,  78898.65849005,  81338.64500545,  80123.08855262,
        71844.76569222,  67347.24678121,  77391.7301448 ,  73131.76979173,
        70518.67618741,    837.67185097, 321130.7696396

In [65]:
ot_fsv_to_start = (sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/fsv_ot_start_percent
rec_fsv_to_start = (sum(list(output_df.Optimized)[10:]) - sum(list(output_df.Original)[10:]))*12
total_fsv_to_start = ot_fsv_to_start + rec_fsv_to_start

(ot_opt, rec_opt) = objective_fn_forecast(output_df.Optimized)
(ot, rec) = objective_fn_forecast(output_df.Month_Orig)
(ot_new, rec_new) = objective_fn_forecast(output_df.Original)


print(f'Total FSV signed for the month: {total_fsv_to_start/current_month}')
print(f'Total OT FSV signed for the month: {ot_fsv_to_start/current_month}')
print(f'Total Rec FSV signed for the month: {rec_fsv_to_start/current_month}')
print('--------------------------------------------------------------')
print(f'Total FSV launched for the month: {1.07*(sum(list(output_df.Optimized)))}')
# print(f'Total OT FSV launched for the month: {ot_fsv_to_start}')
# print(f'Total Rec FSV launched for the month: {rec_fsv_to_start}')
print('--------------------------------------------------------------')
print(f'New OT FSV applicable for month: {ot_fsv_to_start*fsv_ot_start_percent}')
print(f'New Rec FSV applicable for month: {rec_fsv_to_start/12}')
print('--------------------------------------------------------------')
print(f'Carryover OT IR for month: {ot_new - ot}')
print(f'Carryover Rec IR applicable for month: {(rec_new - rec)}')
print('--------------------------------------------------------------')
print(f'New OT IR for month: {ot_opt - ot_new}')
print(f'New Rec IR applicable for month: {0.3*(rec_opt - rec_new)}')
print('--------------------------------------------------------------')


cfo_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
cpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
fpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
bkp_fsv_to_start = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
tax_fsv_to_start = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12

fixed_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fixed.value])[0:5]) - sum(list(output_df.Original[Mix.fixed.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fixed.value])[5:]) - sum(list(output_df.Original[Mix.fixed.value])[5:]))*12
hourly_fsv_to_start = ((sum(list(output_df.Optimized[Mix.hourly.value])[0:5]) - sum(list(output_df.Original[Mix.hourly.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.hourly.value])[5:]) - sum(list(output_df.Original[Mix.hourly.value])[5:]))*12


print(f'OT Mix: {int(np.round(ot_fsv_to_start*100/total_fsv_to_start))}')
print(f'Rec Mix: {int(np.round(rec_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')
print(f'CFO Mix: {int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))}, CFO FSV: {cfo_fsv_to_start}')
print(f'CPA Mix: {int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'FPA Mix: {int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'BKP Mix: {int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))}')
print(f'TAX Mix: {int(np.round(tax_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')

print(f'Fixed Mix: {int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))}')
print(f'Hourly Mix: {int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))}')


output_df['fsv_annualized'] = output_df.apply(lambda x: (x.increment/fsv_ot_start_percent)/current_month if x.frequency == 'one_time' else (x.increment*12)/current_month, axis=1)
output_df.fsv_annualized.to_clipboard()


Total FSV signed for the month: 4727812.4999999935
Total OT FSV signed for the month: 709171.8749999987
Total Rec FSV signed for the month: 4018640.6249999944
--------------------------------------------------------------
Total FSV launched for the month: 3059217.446304715
--------------------------------------------------------------
New OT FSV applicable for month: 248210.15624999953
New Rec FSV applicable for month: 167443.35937499977
--------------------------------------------------------------
Carryover OT IR for month: 387434.65291387273
Carryover Rec IR applicable for month: 1126986.062572276
--------------------------------------------------------------
New OT IR for month: 185172.4393923852
New Rec IR applicable for month: 131372.65939124319
--------------------------------------------------------------
OT Mix: 15
Rec Mix: 85
--------------------------------------------------------------
CFO Mix: 25, CFO FSV: 590976.5625000002
CPA Mix: 22
FPA Mix: 5
BKP Mix: 40
TAX Mix: 8
---

## December

In [67]:
## Constraints

x_0_date = '2021-12-01'
x_0 = x_0_transform.filter(like = x_0_date, axis=0).values[0].tolist()

x_0_orig = x_0
churn_rate = 0.9

# adding in projects already launched
x_0 = [(x_0[idx] + november_additions[idx]) if idx <10 else (x_0[idx] + (churn_rate**6)*june_additions[idx] + (churn_rate**5)*july_additions[idx] 
                                                              + (churn_rate**4)*august_additions[idx] + (churn_rate**3)*september_additions[idx] 
                                                            + (churn_rate**2)*october_additions[idx] + (churn_rate**1)*november_additions[idx]) for idx,value in enumerate(x_0) ]

# adding projects not yet launched
x_0 = [(x_0[idx] + september_additions[idx]*three_months_back/current_month + october_additions[idx]*two_months_back/current_month 
        + november_additions[idx]*previous_month/current_month) for idx,value in enumerate(x_0) ]

fsv_target = 5678125 #December
fsv_previous_month = 6493750 #November
fsv_2_months_back = 5743750 #October
fsv_3_months_back = 5209375 #September

one_time_mix_lower = 0.05
one_time_mix_upper = 0.15

recurring_mix_lower = 0.5
recurring_mix_upper = 0.8

cfo_mix_lower = 0.25
cfo_mix_upper = 0.35

cpa_mix_lower = 0.21
cpa_mix_upper = 0.32

fpa_mix_lower = 0.03
fpa_mix_upper = 0.1

bkp_mix_lower = 0.5
bkp_mix_upper = 0.26

tax_mix_lower = 0.05
tax_mix_upper = 0.08

fixed_mix_lower = 0.3
fixed_mix_upper = 0.7

hourly_mix_lower = 0.15
hourly_mix_upper = 0.4

#Realization ratios
current_month = 0.5
previous_month = 0.25
two_months_back = 0.1
three_months_back = 0.05


fsv_ot_start_percent = 0.7

target_achieve_factor = 0.9
target_miss_factor = 0.8


fsv_to_start = current_month*fsv_target #+ previous_month*fsv_previous_month + two_months_back*fsv_2_months_back + three_months_back*fsv_3_months_back
fsv_to_start = fsv_to_start*target_achieve_factor

upper_limit_one_time_pr_lower = fsv_to_start*one_time_mix_lower


def total_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return fsv_to_start - (ot_annual_fsv + rec_annual_fsv)

def total_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (ot_annual_fsv + rec_annual_fsv) - target_miss_factor*fsv_to_start

def one_time_fsv_constraint(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return (fsv_to_start)*one_time_mix_upper - ot_annual_fsv

def one_time_fsv_constraint_lower(x):
    ot_addition = sum(x[0:10]) - sum(x_0[0:10])
    rec_addition = sum(x[10:]) - sum(x_0[10:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    return ot_annual_fsv - (fsv_to_start)*one_time_mix_lower 

def hourly_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return fsv_to_start*hourly_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def hourly_fsv_constraint_lower(x):
    
    current_fsv = sum([val for idx, val in enumerate(x_0) if idx%2 != 0])
    additional_fsv = (fsv_to_start*one_time_mix_upper*fsv_ot_start_percent + fsv_to_start*recurring_mix_upper/12)*hourly_mix_upper
    
    ot_addition = sum(np.array(x)[Mix.hourly.value][0:5]) - sum(np.array(x_0)[Mix.hourly.value][0:5])
    rec_addition = sum(np.array(x)[Mix.hourly.value][5:]) - sum(np.array(x_0)[Mix.hourly.value][5:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12
    
    
    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*hourly_mix_lower

def cfo_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*cfo_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def cfo_fsv_constraint_lower(x):
    ot_addition = sum(np.array(x)[Mix.cfo.value][0:2]) - sum(np.array(x_0)[Mix.cfo.value][0:2])
    rec_addition = sum(np.array(x)[Mix.cfo.value][2:]) - sum(np.array(x_0)[Mix.cfo.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return (ot_annual_fsv + rec_annual_fsv) - fsv_to_start*cfo_mix_lower

def fpa_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.fpa.value][0:2]) - sum(np.array(x_0)[Mix.fpa.value][0:2])
    rec_addition = sum(np.array(x)[Mix.fpa.value][2:]) - sum(np.array(x_0)[Mix.fpa.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*fpa_mix_upper - (ot_annual_fsv + rec_annual_fsv)

def tax_fsv_constraint(x):
    ot_addition = sum(np.array(x)[Mix.tax.value][0:2]) - sum(np.array(x_0)[Mix.tax.value][0:2])
    rec_addition = sum(np.array(x)[Mix.tax.value][2:]) - sum(np.array(x_0)[Mix.tax.value][2:])
    
    ot_annual_fsv = ot_addition/fsv_ot_start_percent
    rec_annual_fsv = rec_addition*12

    return fsv_to_start*tax_mix_upper - (ot_annual_fsv + rec_annual_fsv)  
    
total_fsv_cons = {'type':'ineq', 'fun':total_fsv_constraint}
total_fsv_cons_lower = {'type':'ineq', 'fun':total_fsv_constraint_lower}
ot_fsv_cons = {'type':'ineq', 'fun':one_time_fsv_constraint}
ot_fsv_cons_lower = {'type':'ineq', 'fun':one_time_fsv_constraint_lower}

hourly_fsv_cons = {'type':'ineq', 'fun':hourly_fsv_constraint}
hourly_fsv_cons_lower = {'type':'ineq', 'fun':hourly_fsv_constraint_lower}
cfo_fsv_cons = {'type':'ineq', 'fun':cfo_fsv_constraint}
cfo_fsv_cons_lower = {'type':'ineq', 'fun':cfo_fsv_constraint_lower}
fpa_fsv_cons = {'type':'ineq', 'fun':fpa_fsv_constraint}
tax_fsv_cons = {'type':'ineq', 'fun':tax_fsv_constraint}

constraints = [total_fsv_cons, total_fsv_cons_lower, ot_fsv_cons, ot_fsv_cons_lower, hourly_fsv_cons, hourly_fsv_cons_lower, cfo_fsv_cons, cfo_fsv_cons_lower, fpa_fsv_cons, tax_fsv_cons ]

bounds = ()
for idx, val in enumerate(x_0):
    bound_val = (x_0[idx] *1.05,800000)
    bounds = bounds + (bound_val,)
    
trend = 41
print_mode = 0
opt_method = 'COBYLA'
# opt_method = 'L-BFGS-B'
opt_method = 'SLSQP'
sol = minimize(objective_fn, x_0, method=opt_method, bounds = bounds, constraints=constraints, options={'maxiter': 500000, 'disp': True ,'eps' : 0.5})


output_df = pd.DataFrame([sol.x, x_0, x_0_orig]).T
output_df.columns = ['Optimized', 'Original', 'Month_Orig']
output_df['Optimized'] = output_df.apply(lambda x: x.Optimized if x.Optimized > x.Original else x.Original, axis=1)
output_df['increment'] = output_df['Optimized'] - output_df['Original']
output_df['type'] = output_columns
output_df['frequency'] = output_frequency
output_df['sl'] = output_sl
output_df['rate'] = output_rate_type
december_additions = list(1*output_df['increment'])
print_mode = 0
print(f'New FSV for the month: {1.11*(sum(list(output_df.Optimized)) - sum(list(output_df.Original)))}')
print(f'New IR for the month: {-1*1000*(objective_fn(list(output_df.Optimized)) - objective_fn(list(output_df.Original)))}')
print(f'Total FSV for the month: {1.11*1.05*sum(list(output_df.Optimized))}')
print(sol)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3537.306169441311
            Iterations: 124
            Function evaluations: 2584
            Gradient evaluations: 123
New FSV for the month: 498702.62109399866
New IR for the month: 736110.1017289107
Total FSV for the month: 3494578.0388959795
     fun: -3537.306169441311
     jac: array([-0.00072424, -0.00075135, -0.00079324, -0.00080931, -0.00067369,
       -0.00065943, -0.00074484, -0.00071444, -0.00079758,  0.01863323,
       -0.0029323 , -0.00246298, -0.00315825, -0.00255002, -0.002076  ,
       -0.00093181, -0.00368283, -0.00270633, -0.0047141 , -0.00404007])
 message: 'Optimization terminated successfully'
    nfev: 2584
     nit: 124
    njev: 123
  status: 0
 success: True
       x: array([ 83991.03146045,  83327.58216414,  87570.31030358,  86530.65099014,
        73693.25440035,  68398.48239387,  81917.55176186,  76093.66543575,
        70268.05129987,    648.22860944, 333892.00660

In [68]:
ot_fsv_to_start = (sum(list(output_df.Optimized)[0:10]) - sum(list(output_df.Original)[0:10]))/fsv_ot_start_percent
rec_fsv_to_start = (sum(list(output_df.Optimized)[10:]) - sum(list(output_df.Original)[10:]))*12
total_fsv_to_start = ot_fsv_to_start + rec_fsv_to_start

(ot_opt, rec_opt) = objective_fn_forecast(output_df.Optimized)
(ot, rec) = objective_fn_forecast(output_df.Month_Orig)
(ot_new, rec_new) = objective_fn_forecast(output_df.Original)


print(f'Total FSV signed for the month: {total_fsv_to_start/current_month}')
print(f'Total OT FSV signed for the month: {ot_fsv_to_start/current_month}')
print(f'Total Rec FSV signed for the month: {rec_fsv_to_start/current_month}')
print('--------------------------------------------------------------')
print(f'Total FSV launched for the month: {1.07*(sum(list(output_df.Optimized)))}')
# print(f'Total OT FSV launched for the month: {ot_fsv_to_start}')
# print(f'Total Rec FSV launched for the month: {rec_fsv_to_start}')
print('--------------------------------------------------------------')
print(f'New OT FSV applicable for month: {ot_fsv_to_start*fsv_ot_start_percent}')
print(f'New Rec FSV applicable for month: {rec_fsv_to_start/12}')
print('--------------------------------------------------------------')
print(f'Carryover OT IR for month: {ot_new - ot}')
print(f'Carryover Rec IR applicable for month: {(rec_new - rec)}')
print('--------------------------------------------------------------')
print(f'New OT IR for month: {ot_opt - ot_new}')
print(f'New Rec IR applicable for month: {0.27*(rec_opt - rec_new)}')
print('--------------------------------------------------------------')


cfo_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cfo.value])[0:2]) - sum(list(output_df.Original[Mix.cfo.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cfo.value])[2:]) - sum(list(output_df.Original[Mix.cfo.value])[2:]))*12
cpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.cpa.value])[0:2]) - sum(list(output_df.Original[Mix.cpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.cpa.value])[2:]) - sum(list(output_df.Original[Mix.cpa.value])[2:]))*12
fpa_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fpa.value])[0:2]) - sum(list(output_df.Original[Mix.fpa.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fpa.value])[2:]) - sum(list(output_df.Original[Mix.fpa.value])[2:]))*12
bkp_fsv_to_start = ((sum(list(output_df.Optimized[Mix.bkp.value])[0:2]) - sum(list(output_df.Original[Mix.bkp.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.bkp.value])[2:]) - sum(list(output_df.Original[Mix.bkp.value])[2:]))*12
tax_fsv_to_start = ((sum(list(output_df.Optimized[Mix.tax.value])[0:2]) - sum(list(output_df.Original[Mix.tax.value])[0:2]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.tax.value])[2:]) - sum(list(output_df.Original[Mix.tax.value])[2:]))*12

fixed_fsv_to_start = ((sum(list(output_df.Optimized[Mix.fixed.value])[0:5]) - sum(list(output_df.Original[Mix.fixed.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.fixed.value])[5:]) - sum(list(output_df.Original[Mix.fixed.value])[5:]))*12
hourly_fsv_to_start = ((sum(list(output_df.Optimized[Mix.hourly.value])[0:5]) - sum(list(output_df.Original[Mix.hourly.value])[0:5]))/fsv_ot_start_percent) +(sum(list(output_df.Optimized[Mix.hourly.value])[5:]) - sum(list(output_df.Original[Mix.hourly.value])[5:]))*12


print(f'OT Mix: {int(np.round(ot_fsv_to_start*100/total_fsv_to_start))}')
print(f'Rec Mix: {int(np.round(rec_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')
print(f'CFO Mix: {int(np.round(cfo_fsv_to_start*100/total_fsv_to_start))}, CFO FSV: {cfo_fsv_to_start}')
print(f'CPA Mix: {int(np.round(cpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'FPA Mix: {int(np.round(fpa_fsv_to_start*100/total_fsv_to_start))}')
print(f'BKP Mix: {int(np.round(bkp_fsv_to_start*100/total_fsv_to_start))}')
print(f'TAX Mix: {int(np.round(tax_fsv_to_start*100/total_fsv_to_start))}')
print('--------------------------------------------------------------')

print(f'Fixed Mix: {int(np.round(fixed_fsv_to_start*100/total_fsv_to_start))}')
print(f'Hourly Mix: {int(np.round(hourly_fsv_to_start*100/total_fsv_to_start))}')


output_df['fsv_annualized'] = output_df.apply(lambda x: (x.increment/fsv_ot_start_percent)/current_month if x.frequency == 'one_time' else (x.increment*12)/current_month, axis=1)
output_df.fsv_annualized.to_clipboard()


Total FSV signed for the month: 5110312.500000698
Total OT FSV signed for the month: 766546.8750006309
Total Rec FSV signed for the month: 4343765.625000067
--------------------------------------------------------------
Total FSV launched for the month: 3208235.5226243655
--------------------------------------------------------------
New OT FSV applicable for month: 268291.4062502208
New Rec FSV applicable for month: 180990.2343750028
--------------------------------------------------------------
Carryover OT IR for month: 392605.68282024347
Carryover Rec IR applicable for month: 1369003.5029464078
--------------------------------------------------------------
New OT IR for month: 200506.38005923416
New Rec IR applicable for month: 144613.0048508126
--------------------------------------------------------------
OT Mix: 15
Rec Mix: 85
--------------------------------------------------------------
CFO Mix: 25, CFO FSV: 638789.0624996831
CPA Mix: 21
FPA Mix: 5
BKP Mix: 41
TAX Mix: 8
-----

In [456]:
x_0

[104048.96671720243,
 101348.49253476867,
 101739.9783208803,
 97822.00934318385,
 88788.06227360874,
 80694.12265368509,
 95666.13942389328,
 88196.48201892107,
 71975.83916333024,
 701.3946716345914,
 443066.4931209836,
 269085.14347219793,
 462212.69449048414,
 249492.72898154563,
 63226.99612476108,
 32402.71935120291,
 719841.0875754156,
 324651.44505734846,
 95179.70461104157,
 41466.25757273978]